In [ ]:
#Extracting the links and names of the scheme

In [ ]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException
)
from webdriver_manager.chrome import ChromeDriverManager

CHECKPOINT_FILE = "myscheme_checkpoint.xlsx"
OUTPUT_FILE = "myscheme_data.xlsx"
TOTAL_PAGES = 329

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode (no UI)
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
         AppleWebKit/537.36 (KHTML, like Gecko) \
         Chrome/91.0.4472.124 Safari/537.36"
    )

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_checkpoint():
    """
    Loads existing data from CHECKPOINT_FILE if it exists,
    and returns a DataFrame plus the last scraped page number.
    """
    if os.path.exists(CHECKPOINT_FILE):
        df = pd.read_excel(CHECKPOINT_FILE)
        if not df.empty:
            # Find the maximum page we've already scraped
            last_page = df['page'].max()
            return df, last_page
    # If file doesn't exist or is empty, start fresh
    return pd.DataFrame(columns=["scheme_name", "scheme_link", "page"]), 0

def save_checkpoint(df):
    """
    Saves the given DataFrame to the checkpoint file.
    Overwrites any existing checkpoint.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)

def scrape_schemes_on_page(driver, current_page):
    """
    Scrapes all scheme names and links on the current page.
    Returns a list of dictionaries with keys: scheme_name, scheme_link, page.
    """
    schemes_data = []

    # Wait briefly for page to load
    time.sleep(2)

    # We look for all <a> elements with href starting with "/schemes/"
    # Each scheme has structure like:
    # <a class="block text-lg ..." href="/schemes/...">
    #     <span>Scheme Name</span>
    # </a>
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href^='/schemes/']"))
        )
    except TimeoutException:
        print(f"Timeout: No schemes found on page {current_page} after waiting 10s.")
        return schemes_data

    a_tags = driver.find_elements(By.CSS_SELECTOR, "a[href^='/schemes/']")
    for a_tag in a_tags:
        link = a_tag.get_attribute("href")  # Full URL after expansion
        # The scheme name is typically in a <span> inside the <a>
        span_elem = a_tag.find_element(By.TAG_NAME, "span")
        scheme_name = span_elem.text.strip() if span_elem else ""

        if scheme_name:
            schemes_data.append({
                "scheme_name": scheme_name,
                "scheme_link": link,
                "page": current_page
            })
    return schemes_data

def click_next_button(driver, current_page):
    """
    Attempts to click the 'Next' pagination button or
    the numeric page link for (current_page + 1).
    Returns True if successful, False otherwise.
    """
    # We'll attempt multiple methods:
    # 1) Numeric link for page (current_page + 1)
    # 2) Rightmost pagination arrow (the next button)

    next_page_num = current_page + 1

    # Approach 1: Numeric page link
    page_link_xpath = f"//li[text()='{next_page_num}'] | //a[text()='{next_page_num}']"
    links = driver.find_elements(By.XPATH, page_link_xpath)
    if links:
        try:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", links[0])
            time.sleep(1)
            links[0].click()
            time.sleep(2)
            return True
        except:
            pass

    # Approach 2: Next arrow button (SVG or last clickable in pagination)
    # This might be an <svg> or a <div> with a cursor-pointer
    try:
        svg_next_buttons = driver.find_elements(
            By.XPATH, "//svg[contains(@class, 'cursor-pointer') or contains(@class, 'text-darkblue-900')]"
        )
        # The site might have multiple SVGs; we assume the last one is "Next"
        if svg_next_buttons:
            button = svg_next_buttons[-1]
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
            time.sleep(1)
            button.click()
            time.sleep(2)
            return True
    except Exception as e:
        print(f"Error clicking next arrow: {e}")

    return False

def main_scraper():
    """
    Main function to scrape all pages from 1 to TOTAL_PAGES,
    resuming from the last scraped page if checkpoint file exists.
    """
    # Load existing data from checkpoint
    checkpoint_df, last_page_scraped = load_checkpoint()

    # Start from the next page if we already scraped something
    start_page = last_page_scraped + 1
    if start_page > TOTAL_PAGES:
        print("All pages already scraped. Exiting.")
        return

    driver = setup_driver()
    driver.get("https://www.myscheme.gov.in/search")
    time.sleep(3)

    current_page = 1
    # If we need to jump to start_page (e.g. we left off at page 15),
    # we repeatedly click next until we reach that page.
    # (We can't do direct URL with ?page=... because the site doesn't load it that way.)
    while current_page < start_page:
        print(f"Jumping to page {start_page}, currently at page {current_page}...")
        success = click_next_button(driver, current_page)
        if success:
            current_page += 1
        else:
            print("Failed to navigate pages while jumping. Exiting.")
            driver.quit()
            return

    # Now we can scrape from start_page to TOTAL_PAGES
    while current_page <= TOTAL_PAGES:
        print(f"\n=== Scraping Page {current_page} ===")

        # Scrape schemes
        page_schemes = scrape_schemes_on_page(driver, current_page)
        print(f"Found {len(page_schemes)} schemes on page {current_page}")

        # Append new data to checkpoint DataFrame
        new_data_df = pd.DataFrame(page_schemes)
        checkpoint_df = pd.concat([checkpoint_df, new_data_df], ignore_index=True)

        # Remove duplicates by scheme_link or scheme_name
        checkpoint_df.drop_duplicates(subset=["scheme_link", "scheme_name"], inplace=True)

        # Save checkpoint after every page
        save_checkpoint(checkpoint_df)
        print(f"Checkpoint saved. Total schemes so far: {len(checkpoint_df)}")

        # If we're not at the last page, click next
        if current_page < TOTAL_PAGES:
            success = click_next_button(driver, current_page)
            if not success:
                print("Could not click next. Ending scraping early.")
                break
        current_page += 1

    driver.quit()

    # Finally, save everything to the final output file
    checkpoint_df.to_excel(OUTPUT_FILE, index=False)
    print(f"\nScraping complete! Total unique schemes: {len(checkpoint_df)}")
    print(f"Data saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main_scraper()

In [ ]:
#Extracting the details of each scheme

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import random

# File paths
INPUT_FILE = "myscheme_data.xlsx"  # Your existing file with scheme links
OUTPUT_FILE = "myscheme_details.xlsx"  # New file for scheme details
CHECKPOINT_FILE = "details_checkpoint.xlsx"  # Checkpoint file for tracking progress

# Maximum number of retries for failed requests
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
         AppleWebKit/537.36 (KHTML, like Gecko) \
         Chrome/91.0.4472.124 Safari/537.36"
    )

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_checkpoint():
    """
    Load the checkpoint file to track progress.
    Returns a DataFrame and the last processed index.
    """
    if os.path.exists(CHECKPOINT_FILE):
        details_df = pd.read_excel(CHECKPOINT_FILE)
        # Find the last processed scheme by looking at the index column
        if not details_df.empty and 'original_index' in details_df.columns:
            last_processed = details_df['original_index'].max()
            return details_df, last_processed
    # If file doesn't exist or is empty, start fresh
    return pd.DataFrame(columns=["scheme_name", "scheme_link", "details_text", "original_index"]), -1

def save_checkpoint(df):
    """
    Save current progress to the checkpoint file.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)
    print(f"Checkpoint saved with {len(df)} schemes.")

def extract_details_section(driver, url):
    """
    Extract the 'Details' section content from a scheme page.
    Returns the extracted text or None if not found.
    """
    try:
        driver.get(url)
        # Wait for page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        # First, find the "Details" heading which indicates the section
        # This is based on the structure seen in the screenshots
        # The details section might be inside a div with id="details" or have a specific heading
        
        # Attempt to find the Details section content using various selectors
        selectors = [
            "#details .markdown-options", 
            "h2:contains('Details') + div", 
            "div.markdown-options", 
            ".markdown-options p",
            "h2:contains('Details') ~ div"
        ]

        for selector in selectors:
            try:
                # Wait for the selector to be present
                element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, selector))
                )
                if element:
                    # Extract text content
                    details_text = element.text.strip()
                    if details_text:
                        return details_text
            except:
                continue
                
        # If the CSS selectors fail, try XPath as a fallback
        try:
            # Looking for elements with text content after a heading with "Details"
            details_element = driver.find_element(By.XPATH, "//h2[contains(text(), 'Details')]/following-sibling::div[1]")
            if details_element:
                return details_element.text.strip()
        except:
            pass
            
        # If all attempts fail, try to get any text content in the page's main content area
        main_content = driver.find_element(By.XPATH, "//main")
        if main_content:
            paragraphs = main_content.find_elements(By.TAG_NAME, "p")
            if paragraphs:
                return "\n".join([p.text.strip() for p in paragraphs])
                
        return "No details section found"
        
    except Exception as e:
        print(f"Error extracting details: {e}")
        return None

def main():
    """
    Main function to process scheme links and extract details.
    """
    # Load the scheme links from the input file
    if not os.path.exists(INPUT_FILE):
        print(f"Input file {INPUT_FILE} not found.")
        return
        
    schemes_df = pd.read_excel(INPUT_FILE)
    
    # Check if the input file has the required columns
    if 'scheme_name' not in schemes_df.columns or 'scheme_link' not in schemes_df.columns:
        print("Input file missing required columns 'scheme_name' or 'scheme_link'.")
        return
        
    # Load checkpoint data
    details_df, last_processed_index = load_checkpoint()
    
    # Set up the WebDriver
    driver = setup_driver()
    
    # Reset index and add original index column if not present
    if 'original_index' not in schemes_df.columns:
        schemes_df['original_index'] = schemes_df.index
    
    # Start from where we left off
    start_index = last_processed_index + 1
    
    # Process schemes
    total_schemes = len(schemes_df)
    for idx, row in schemes_df.iloc[start_index:].iterrows():
        scheme_name = row['scheme_name']
        scheme_link = row['scheme_link']
        original_index = row['original_index'] if 'original_index' in row else idx
        
        print(f"\n[{idx+1}/{total_schemes}] Processing: {scheme_name}")
        
        # Try to extract details with retries
        details_text = None
        retries = 0
        
        while details_text is None and retries < MAX_RETRIES:
            try:
                details_text = extract_details_section(driver, scheme_link)
                if not details_text:
                    details_text = "No details found"
            except Exception as e:
                print(f"Attempt {retries+1} failed: {e}")
                time.sleep(2 + random.random() * 3)  # Random delay between 2-5 seconds
                retries += 1
        
        # If all retries failed
        if details_text is None:
            details_text = "Failed to extract details"
        
        # Add to details DataFrame
        new_row = {
            "scheme_name": scheme_name,
            "scheme_link": scheme_link,
            "details_text": details_text,
            "original_index": original_index
        }
        
        details_df = pd.concat([details_df, pd.DataFrame([new_row])], ignore_index=True)
        
        # Save checkpoint every 10 schemes
        if (idx + 1) % 10 == 0:
            save_checkpoint(details_df)
            print(f"Progress: {idx+1}/{total_schemes} schemes processed.")
            
        # Add a random delay to avoid detection
        time.sleep(1 + random.random() * 2)  # Random delay between 1-3 seconds
    
    # Close the WebDriver
    driver.quit()
    
    # Save final results
    details_df.to_excel(OUTPUT_FILE, index=False)
    save_checkpoint(details_df)
    
    print(f"\nExtraction complete! Processed {len(details_df)} schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the missing details of each scheme

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import random

# File paths
INPUT_FILE = "myscheme_details.xlsx"  # Your existing file with scheme links and partial details
OUTPUT_FILE = "myscheme_details_complete.xlsx"  # Updated file with complete details
CHECKPOINT_FILE = "details_retry_checkpoint.xlsx"  # Checkpoint file for tracking progress

# Maximum number of retries for failed requests
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
         AppleWebKit/537.36 (KHTML, like Gecko) \
         Chrome/91.0.4472.124 Safari/537.36"
    )

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def extract_details_section(driver, url):
    """
    Extract the 'Details' section content from a scheme page.
    Enhanced version with more precise selectors and wait times.
    """
    try:
        driver.get(url)
        # Wait for page to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        # Add a fixed delay to ensure JavaScript rendering completes
        time.sleep(3)
        
        # First try to find the Details heading and then look for content after it
        try:
            # Look for the active tab or section that would contain "Details"
            details_tab = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//h2[text()='Details'] | //button[text()='Details'] | //div[text()='Details'] | //a[text()='Details']"))
            )
            # Click on the Details tab if it's not already active
            if details_tab.tag_name.lower() in ['button', 'a'] and details_tab.is_displayed() and details_tab.is_enabled():
                driver.execute_script("arguments[0].click();", details_tab)
                time.sleep(2)  # Wait for content to load after click
        except:
            # If we can't find or click a Details tab, continue with direct content extraction
            pass
        
        # Multiple approaches to find the content, from most specific to most general
        selectors = [
            # Direct navigation to the details section
            "//div[@id='details']//p | //div[@id='details']//div[contains(@class, 'markdown-options')]",
            
            # Based on your HTML structure in Image 3
            "//div[contains(@class, 'flex-1')]//div[contains(@class, 'markdown-options')]//div[contains(@class, 'mb-2')]",
            
            # Find content after a details heading
            "//h2[text()='Details']/following-sibling::div[1]//p | //h2[text()='Details']/following-sibling::div[1]",
            
            # Look for specific data-slate attributes as shown in Image 2
            "//span[@data-slate-string='true']",
            
            # More generic approaches
            "//div[contains(@class, 'flex-1')]//p",
            "//main//p"
        ]
        
        for selector in selectors:
            try:
                elements = driver.find_elements(By.XPATH, selector)
                if elements:
                    texts = [el.text.strip() for el in elements if el.text.strip()]
                    if texts:
                        return "\n".join(texts)
            except Exception as e:
                print(f"Selector {selector} failed: {e}")
                continue
        
        # Last resort: capture any visible text in the main content area
        try:
            main_content = driver.find_element(By.TAG_NAME, "main")
            paragraphs = main_content.find_elements(By.TAG_NAME, "p")
            if paragraphs:
                texts = [p.text.strip() for p in paragraphs if p.text.strip()]
                if texts:
                    return "\n".join(texts)
        except:
            pass
            
        # If we still have nothing, try to get text directly using JavaScript
        try:
            script = """
            // Find Details section using various approaches
            let detailsContent = '';
            
            // Approach 1: Find by ID
            const detailsSection = document.getElementById('details');
            if (detailsSection) {
                detailsContent = detailsSection.innerText;
            }
            
            // Approach 2: Find by heading
            if (!detailsContent) {
                const headings = Array.from(document.querySelectorAll('h2, h3'));
                const detailsHeading = headings.find(h => h.innerText.includes('Details'));
                if (detailsHeading) {
                    let nextElem = detailsHeading.nextElementSibling;
                    while (nextElem && !['H2', 'H3'].includes(nextElem.tagName)) {
                        detailsContent += nextElem.innerText + '\\n';
                        nextElem = nextElem.nextElementSibling;
                    }
                }
            }
            
            // Approach 3: Find by tab content
            if (!detailsContent) {
                const tabContents = document.querySelectorAll('.tab-content, [role="tabpanel"]');
                for (const tab of tabContents) {
                    if (tab.innerText.length > 20) {  // Assume content has reasonable length
                        detailsContent = tab.innerText;
                        break;
                    }
                }
            }
            
            return detailsContent.trim();
            """
            js_result = driver.execute_script(script)
            if js_result:
                return js_result
        except Exception as e:
            print(f"JavaScript extraction failed: {e}")
            
        # If still nothing found
        return "No details found"
        
    except Exception as e:
        print(f"Error extracting details: {e}")
        return "Error: " + str(e)

def main():
    """
    Main function to process scheme links with "No details found" and extract details.
    """
    # Load the scheme details from the input file
    if not os.path.exists(INPUT_FILE):
        print(f"Input file {INPUT_FILE} not found.")
        return
        
    schemes_df = pd.read_excel(INPUT_FILE)
    
    # Check if the input file has the required columns
    required_columns = ['scheme_name', 'scheme_link', 'details_text', 'original_index']
    if not all(col in schemes_df.columns for col in required_columns):
        print(f"Input file missing required columns. Required: {required_columns}")
        return
    
    # Filter only rows with "No details found"
    no_details_df = schemes_df[schemes_df['details_text'].str.contains('No details found', na=False)]
    has_details_df = schemes_df[~schemes_df['details_text'].str.contains('No details found', na=False)]
    
    print(f"Found {len(no_details_df)} schemes with 'No details found'")
    print(f"{len(has_details_df)} schemes already have details")
    
    if len(no_details_df) == 0:
        print("No schemes need processing. Exiting.")
        return
    
    # Set up the WebDriver
    driver = setup_driver()
    
    # Process schemes with no details
    updated_rows = []
    total_to_process = len(no_details_df)
    
    for idx, row in no_details_df.iterrows():
        scheme_name = row['scheme_name']
        scheme_link = row['scheme_link']
        original_index = row['original_index']
        
        print(f"\n[{idx+1}/{total_to_process}] Processing: {scheme_name}")
        
        # Try to extract details with retries
        details_text = None
        retries = 0
        
        while details_text is None and retries < MAX_RETRIES:
            try:
                details_text = extract_details_section(driver, scheme_link)
                if not details_text or details_text == "No details found":
                    print(f"Attempt {retries+1}: Still no details found")
                    retries += 1
                    time.sleep(2 + random.random() * 3)  # Random delay
                    details_text = None  # Reset to trigger retry
                else:
                    print(f"Successfully extracted details: {len(details_text)} characters")
            except Exception as e:
                print(f"Attempt {retries+1} failed: {e}")
                time.sleep(2 + random.random() * 3)
                retries += 1
        
        # If all retries failed
        if details_text is None:
            details_text = "No details found after multiple attempts"
        
        # Update the row
        updated_row = row.copy()
        updated_row['details_text'] = details_text
        updated_rows.append(updated_row)
        
        # Save checkpoint every 5 schemes
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            # Create temporary DataFrame with processed rows
            processed_df = pd.DataFrame(updated_rows)
            # Merge with schemes that already have details
            temp_df = pd.concat([has_details_df, processed_df])
            temp_df.to_excel(CHECKPOINT_FILE, index=False)
            print(f"Checkpoint saved. Processed {idx+1}/{total_to_process} no-details schemes.")
            
        # Add a random delay to avoid detection
        time.sleep(1 + random.random() * 2)
    
    # Close the WebDriver
    driver.quit()
    
    # Combine updated rows with rows that already had details
    final_df = pd.concat([has_details_df, pd.DataFrame(updated_rows)])
    
    # Sort by original index to maintain the original order
    final_df = final_df.sort_values('original_index')
    
    # Save final results
    final_df.to_excel(OUTPUT_FILE, index=False)
    
    print(f"\nExtraction complete! Updated {len(updated_rows)} schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the benefit section of each scheme

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import random

# File paths
INPUT_FILE = "myscheme_details_complete.xlsx"  # Your existing file with scheme links and details
OUTPUT_FILE = "myscheme_details_with_benefits.xlsx"  # Updated file with benefits added
CHECKPOINT_FILE = "benefits_retry_checkpoint.xlsx"  # Checkpoint file for tracking progress

# Maximum number of retries for failed requests
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
         AppleWebKit/537.36 (KHTML, like Gecko) \
         Chrome/91.0.4472.124 Safari/537.36"
    )

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def check_for_table(driver):
    """
    Check if there's a table in the Benefits section.
    Returns True if a table is found, False otherwise.
    """
    # First check for tables in the active tab or benefits section
    specific_selectors = [
        "//div[contains(@class, 'active')]//table",
        "//div[@id='benefits']//table",
        "//div[contains(@class, 'benefits')]//table",
        "//h2[contains(text(), 'Benefits')]/following-sibling::div//table",
        "//section[contains(@class, 'benefits')]//table"
    ]
    
    for selector in specific_selectors:
        try:
            tables = driver.find_elements(By.XPATH, selector)
            if tables and len(tables) > 0 and tables[0].is_displayed():
                print(f"Table detected in benefits section using selector: {selector}")
                return True
        except Exception as e:
            continue
    
    # JavaScript detection - more comprehensive
    script = """
    function detectBenefitsTable() {
        // Find benefits section first
        const benefitsSection = document.getElementById('benefits') || 
                               document.querySelector('.benefits') ||
                               document.querySelector('[data-section="benefits"]');
        
        // Check if benefits section contains a table
        if (benefitsSection && benefitsSection.querySelector('table')) {
            return true;
        }
        
        // Check if any heading contains "Benefits" and next siblings contain table
        const headings = Array.from(document.querySelectorAll('h1, h2, h3, h4, h5'));
        const benefitsHeading = headings.find(h => h.innerText.toLowerCase().includes('benefits'));
        
        if (benefitsHeading) {
            // Look through next siblings until we hit another heading
            let currentNode = benefitsHeading.nextElementSibling;
            while (currentNode && !['H1', 'H2', 'H3', 'H4', 'H5'].includes(currentNode.tagName)) {
                if (currentNode.tagName === 'TABLE' || currentNode.querySelector('table')) {
                    return true;
                }
                currentNode = currentNode.nextElementSibling;
            }
        }
        
        // Check active tabs for tables
        const activeTabs = document.querySelectorAll('[role="tabpanel"].active, .tab-content.active, .tab-pane.active');
        for (const tab of activeTabs) {
            if (tab.querySelector('table')) {
                return true;
            }
        }
        
        return false;
    }
    
    return detectBenefitsTable();
    """
    
    try:
        has_table = driver.execute_script(script)
        if has_table:
            print("Table detected via JavaScript")
            return True
    except Exception as e:
        print(f"JavaScript table detection failed: {e}")
    
    return False

def extract_table_contents(driver):
    """
    Extract the contents of tables in the Benefits section and format them nicely.
    """
    table_data = []
    
    # Use various selectors to find tables in benefits sections
    table_selectors = [
        "//div[contains(@class, 'active')]//table",
        "//div[@id='benefits']//table",
        "//div[contains(@class, 'benefits')]//table",
        "//h2[contains(text(), 'Benefits')]/following-sibling::div//table",
        "//section[contains(@class, 'benefits')]//table"
    ]
    
    for selector in table_selectors:
        try:
            tables = driver.find_elements(By.XPATH, selector)
            for table in tables:
                if table.is_displayed():
                    # Try to extract rows
                    rows = table.find_elements(By.TAG_NAME, "tr")
                    if rows:
                        for row in rows:
                            # Extract header cells and data cells
                            headers = row.find_elements(By.TAG_NAME, "th")
                            cells = row.find_elements(By.TAG_NAME, "td")
                            
                            if headers and len(headers) > 0:
                                # It's a header row
                                header_text = " | ".join([h.text.strip() for h in headers if h.text.strip()])
                                if header_text:
                                    table_data.append(f"Header: {header_text}")
                            elif cells and len(cells) > 0:
                                # It's a data row
                                cell_text = " | ".join([c.text.strip() for c in cells if c.text.strip()])
                                if cell_text:
                                    table_data.append(f"• {cell_text}")
                    
                    # If we found data, return it
                    if table_data:
                        return "\n".join(table_data)
        except Exception as e:
            print(f"Error extracting table with selector {selector}: {e}")
    
    # Try using JavaScript for more complex tables
    script = """
    function extractTableData() {
        // Find tables in benefits sections
        const tables = [];
        
        // Find benefits section first
        const benefitsSections = [
            document.getElementById('benefits'),
            ...Array.from(document.querySelectorAll('.benefits, [data-section="benefits"]'))
        ].filter(Boolean);
        
        // Add tables from benefits sections
        benefitsSections.forEach(section => {
            section.querySelectorAll('table').forEach(table => tables.push(table));
        });
        
        // Add tables from active tabs
        document.querySelectorAll('[role="tabpanel"].active, .tab-content.active, .tab-pane.active')
            .forEach(tab => tab.querySelectorAll('table').forEach(table => tables.push(table)));
        
        // Add tables after Benefits headings
        const headings = Array.from(document.querySelectorAll('h1, h2, h3, h4, h5'))
            .filter(h => h.innerText.toLowerCase().includes('benefits'));
            
        headings.forEach(heading => {
            let element = heading.nextElementSibling;
            while (element && !element.tagName.match(/^H[1-5]$/)) {
                element.querySelectorAll('table').forEach(table => tables.push(table));
                element = element.nextElementSibling;
            }
        });
        
        // If no tables found
        if (tables.length === 0) return '';
        
        // Extract data from tables
        let result = [];
        
        tables.forEach((table, tableIndex) => {
            // Add table number if multiple tables
            if (tables.length > 1) {
                result.push(`Table ${tableIndex + 1}:`);
            }
            
            // Get rows
            const rows = table.querySelectorAll('tr');
            rows.forEach((row, rowIndex) => {
                const headerCells = row.querySelectorAll('th');
                const dataCells = row.querySelectorAll('td');
                
                // Process header cells
                if (headerCells.length > 0) {
                    const headerText = Array.from(headerCells)
                        .map(cell => cell.innerText.trim())
                        .filter(text => text)
                        .join(' | ');
                    
                    if (headerText) {
                        result.push(`Header: ${headerText}`);
                    }
                }
                
                // Process data cells
                if (dataCells.length > 0) {
                    const cellText = Array.from(dataCells)
                        .map(cell => cell.innerText.trim())
                        .filter(text => text)
                        .join(' | ');
                    
                    if (cellText) {
                        result.push(`• ${cellText}`);
                    }
                }
            });
        });
        
        return result.join('\\n');
    }
    
    return extractTableData();
    """
    
    try:
        js_result = driver.execute_script(script)
        if js_result and len(js_result) > 10:
            return js_result
    except Exception as e:
        print(f"JavaScript table extraction failed: {e}")
    
    return "Table detected but content could not be extracted"

def extract_benefits_section(driver, url):
    """
    Extract the 'Benefits' section content from a scheme page.
    Handles both bullet-point style benefits and extracts tables.
    """
    try:
        driver.get(url)
        # Wait for page to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        
        # Add a fixed delay to ensure JavaScript rendering completes
        time.sleep(3)
        
        # First approach: Look for tabs and click on "Benefits" if it exists
        try:
            # Use a more comprehensive XPath to find Benefits tab with case-insensitive matching
            benefits_tab = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((
                    By.XPATH, 
                    "//a[contains(translate(text(),'benefits','BENEFITS'),'BENEFITS')] | " +
                    "//button[contains(translate(text(),'benefits','BENEFITS'),'BENEFITS')] | " +
                    "//div[contains(translate(text(),'benefits','BENEFITS'),'BENEFITS') and not(ancestor::*[contains(@class, 'content')])] | " +
                    "//span[contains(translate(text(),'benefits','BENEFITS'),'BENEFITS') and not(ancestor::*[contains(@class, 'content')])]"
                ))
            )
            
            # Check if element is clickable before clicking
            if benefits_tab.is_displayed() and benefits_tab.is_enabled():
                # Use JavaScript click for more reliability
                driver.execute_script("arguments[0].click();", benefits_tab)
                print("Clicked on Benefits tab")
                time.sleep(3)  # Wait longer for content to load after click
        except Exception as e:
            print(f"Could not find or click Benefits tab: {e}")
        
        # Check for tables in benefits section first
        if check_for_table(driver):
            print("Table found in benefits section, extracting table content...")
            table_content = extract_table_contents(driver)
            if table_content and len(table_content) > 10:
                return table_content

        # Check for section heading first, this is the most reliable indicator
        try:
            # Look for Benefits heading (h1, h2, h3, or h4)
            headings = driver.find_elements(By.XPATH, 
                "//h1[contains(translate(text(),'Benefits','BENEFITS'),'BENEFITS')] | " +
                "//h2[contains(translate(text(),'Benefits','BENEFITS'),'BENEFITS')] | " +
                "//h3[contains(translate(text(),'Benefits','BENEFITS'),'BENEFITS')] | " +
                "//h4[contains(translate(text(),'Benefits','BENEFITS'),'BENEFITS')]")
            
            if headings:
                # Get the first heading that contains "Benefits"
                benefits_heading = headings[0]
                print(f"Found Benefits heading: {benefits_heading.text}")
                
                # Try to find the section content based on the heading
                section_content = None
                
                # Method 1: Get sibling div or section content
                try:
                    # Look for the next siblings or content that follows the heading
                    next_element = driver.execute_script("""
                        function getNextElements(element, maxElements = 5) {
                            let result = '';
                            let currentElement = element;
                            let count = 0;
                            
                            while (currentElement.nextElementSibling && count < maxElements) {
                                let nextElem = currentElement.nextElementSibling;
                                
                                // Stop if we hit another heading or a new section
                                if (nextElem.tagName.startsWith('H') || 
                                    nextElem.className.includes('section') ||
                                    (nextElem.innerText && (
                                        nextElem.innerText.includes('Eligibility') || 
                                        nextElem.innerText.includes('Documents Required') || 
                                        nextElem.innerText.includes('Application Process')
                                    ))) {
                                    break;
                                }
                                
                                // Add element content if it's visible and has text
                                if (nextElem.offsetParent !== null && nextElem.innerText.trim()) {
                                    result += nextElem.innerText + '\\n';
                                }
                                
                                currentElement = nextElem;
                                count++;
                            }
                            
                            return result.trim();
                        }
                        return getNextElements(arguments[0]);
                    """, benefits_heading)
                    
                    if next_element and len(next_element) > 10:
                        section_content = next_element
                except Exception as e:
                    print(f"Error getting next elements: {e}")
                
                # Method 2: Find nearest container div
                if not section_content:
                    try:
                        # Get parent div or section that might contain content
                        parent_container = driver.execute_script("""
                            function findContentContainer(element) {
                                // Get parent container that likely contains all content
                                let current = element;
                                
                                // Look up for a container (div, section) that would contain content
                                while (current && current.tagName !== 'BODY') {
                                    current = current.parentElement;
                                    
                                    // Skip tiny containers
                                    if (current && current.innerText.length < 20) continue;
                                    
                                    // If we find a suitable container with content
                                    if (current && 
                                        (current.tagName === 'DIV' || current.tagName === 'SECTION') &&
                                        current.innerText.length > 50) {
                                        
                                        // Extract only relevant part
                                        const fullText = current.innerText;
                                        const headingText = element.innerText;
                                        const startPos = fullText.indexOf(headingText);
                                        
                                        if (startPos >= 0) {
                                            // Find where the next section likely begins
                                            const endPos = fullText.indexOf('Eligibility', startPos + headingText.length);
                                            if (endPos > startPos) {
                                                return fullText.substring(startPos + headingText.length, endPos).trim();
                                            } else {
                                                // Just get content after heading
                                                return fullText.substring(startPos + headingText.length).trim();
                                            }
                                        }
                                    }
                                }
                                return '';
                            }
                            return findContentContainer(arguments[0]);
                        """, benefits_heading)
                        
                        if parent_container and len(parent_container) > 10:
                            section_content = parent_container
                    except Exception as e:
                        print(f"Error getting parent container: {e}")
                
                # If content was found, return it
                if section_content:
                    return section_content
        except Exception as e:
            print(f"Error processing Benefits heading: {e}")

        # Comprehensive approach to find bullet points
        selectors = [
            # Active tab content with bullet points
            "//div[contains(@class, 'active') or @role='tabpanel']//ul/li",
            
            # Benefits section with bullet points
            "//div[@id='benefits']//ul/li | //div[@id='benefits']//ol/li",
            
            # Content after Benefits heading
            "//h2[contains(text(), 'Benefits')]/following-sibling::ul[1]/li | " +
            "//h2[contains(text(), 'Benefits')]/following-sibling::div//ul/li",
            
            # Markdown formatted lists
            "//div[contains(@class, 'markdown-options')]//ul/li",
            
            # Any bullets within the main content
            "//main//ul/li[contains(@class, 'benefits') or ancestor::*[contains(@class, 'benefits')]]"
        ]
        
        for selector in selectors:
            try:
                bullet_points = driver.find_elements(By.XPATH, selector)
                if bullet_points:
                    texts = [point.text.strip() for point in bullet_points if point.text.strip()]
                    if texts:
                        formatted_text = "\n".join([f"• {text}" for text in texts])
                        return formatted_text
            except Exception as e:
                print(f"Bullet point selector {selector} failed: {e}")
        
        # Try using JavaScript to extract content based on structure
        script = """
        function extractBenefitsContent() {
            // Approach 1: Find by ID or class
            const benefitsSections = [
                document.getElementById('benefits'),
                ...Array.from(document.querySelectorAll('.benefits, .benefit-content, [data-section="benefits"]'))
            ].filter(Boolean);
            
            if (benefitsSections.length > 0) {
                const section = benefitsSections[0];
                
                // Check for bullet points
                const bullets = section.querySelectorAll('li');
                if (bullets.length > 0) {
                    return Array.from(bullets)
                        .map(b => '• ' + b.innerText.trim())
                        .join('\\n');
                }
                
                // Otherwise return full text
                return section.innerText.trim();
            }
            
            // Approach 2: Find by looking for Benefits heading
            const headings = Array.from(document.querySelectorAll('h1, h2, h3, h4, h5'));
            const benefitsHeading = headings.find(h => 
                h.innerText.trim().toLowerCase().includes('benefits')
            );
            
            if (benefitsHeading) {
                // Get content between this heading and next heading
                let content = '';
                let currentNode = benefitsHeading.nextElementSibling;
                
                while (currentNode && 
                      !['H1', 'H2', 'H3', 'H4', 'H5'].includes(currentNode.tagName) &&
                      !currentNode.innerText.includes('Eligibility') &&
                      !currentNode.innerText.includes('Documents Required') &&
                      !currentNode.innerText.includes('Application Process')) {
                    
                    if (currentNode.innerText.trim()) {
                        content += currentNode.innerText.trim() + '\\n';
                    }
                    currentNode = currentNode.nextElementSibling;
                }
                
                return content.trim();
            }
            
            // Approach 3: Look for active tab content
            const activeTabs = document.querySelectorAll('[role="tabpanel"].active, .tab-content.active, .tab-pane.active');
            for (const tab of activeTabs) {
                if (tab.innerText.includes('Benefits') || 
                    tab.previousElementSibling?.innerText.includes('Benefits')) {
                    return tab.innerText.trim();
                }
            }
            
            // Approach 4: Look for content in data-slate-* tags (as seen in your images)
            const slateElements = document.querySelectorAll('[data-slate-string="true"]');
            if (slateElements.length > 0) {
                // Check if any parent has "Benefits" text
                for (const elem of slateElements) {
                    let parent = elem.parentElement;
                    while (parent && parent !== document.body) {
                        if (parent.innerText.toLowerCase().includes('benefits')) {
                            // Found a slate element inside Benefits section
                            const parentSection = parent.closest('div[data-slate-node="element"]');
                            if (parentSection) {
                                return parentSection.innerText;
                            }
                        }
                        parent = parent.parentElement;
                    }
                }
            }
            
            return '';
        }
        
        return extractBenefitsContent();
        """
        
        try:
            js_result = driver.execute_script(script)
            if js_result and len(js_result) > 10:
                return js_result
        except Exception as e:
            print(f"JavaScript extraction failed: {e}")
        
        # Last resort: Find any content that might be benefits
        fallback_script = """
        function findPotentialBenefits() {
            // Look for paragraphs with key benefit phrases
            const benefitPhrases = ['amount', 'compensation', 'grant', 'stipend', 'financial', 'assistance', 'support', 'paid', 'receive'];
            
            const paragraphs = document.querySelectorAll('p');
            let benefitsText = '';
            
            for (const p of paragraphs) {
                const text = p.innerText.toLowerCase();
                if (benefitPhrases.some(phrase => text.includes(phrase))) {
                    benefitsText += '• ' + p.innerText.trim() + '\\n';
                }
            }
            
            // If we found something
            if (benefitsText) return benefitsText;
            
            // Try finding main content
            const mainContent = document.querySelector('main') || document.querySelector('.content') || document.body;
            const mainText = mainContent.innerText;
            
            // Try to find Benefits section within main content
            const benefitsIndex = mainText.indexOf('Benefits');
            if (benefitsIndex >= 0) {
                // Look for next section heading
                const nextSectionIndex = Math.min(
                    ...[
                        mainText.indexOf('Eligibility', benefitsIndex),
                        mainText.indexOf('Documents Required', benefitsIndex),
                        mainText.indexOf('Application Process', benefitsIndex)
                    ].filter(idx => idx > 0)
                );
                
                if (nextSectionIndex > benefitsIndex) {
                    return mainText.substring(benefitsIndex, nextSectionIndex).trim();
                }
            }
            
            return '';
        }
        
        return findPotentialBenefits();
        """
        
        try:
            fallback_result = driver.execute_script(fallback_script)
            if fallback_result and len(fallback_result) > 10:
                return fallback_result
        except Exception as e:
            print(f"Fallback extraction failed: {e}")
        
        return "No benefits information found"
        
    except Exception as e:
        print(f"Error extracting benefits: {e}")
        return "Error: " + str(e)

def main():
    """
    Main function to process scheme links and extract benefits information.
    """
    # Load the scheme details from the input file
    if not os.path.exists(INPUT_FILE):
        print(f"Input file {INPUT_FILE} not found.")
        return
        
    schemes_df = pd.read_excel(INPUT_FILE)
    
    # Check if the input file has the required columns
    required_columns = ['scheme_name', 'scheme_link']
    if not all(col in schemes_df.columns for col in required_columns):
        print(f"Input file missing required columns. Required: {required_columns}")
        return
    
    # Add a benefits_text column if it doesn't exist
    if 'benefits_text' not in schemes_df.columns:
        schemes_df['benefits_text'] = ""
    
    # Filter only rows without benefits information
    no_benefits_df = schemes_df[schemes_df['benefits_text'].isna() | (schemes_df['benefits_text'] == "")]
    has_benefits_df = schemes_df[~(schemes_df['benefits_text'].isna() | (schemes_df['benefits_text'] == ""))]
    
    print(f"Found {len(no_benefits_df)} schemes without benefits information")
    print(f"{len(has_benefits_df)} schemes already have benefits information")
    
    if len(no_benefits_df) == 0:
        print("No schemes need processing. Exiting.")
        return
    
    # Set up the WebDriver
    driver = setup_driver()
    
    # Process schemes without benefits
    updated_rows = []
    total_to_process = len(no_benefits_df)
    
    for idx, row in no_benefits_df.iterrows():
        scheme_name = row['scheme_name']
        scheme_link = row['scheme_link']
        
        # Skip if URL is invalid or empty
        if not scheme_link or not scheme_link.startswith('http'):
            print(f"Skipping invalid URL for: {scheme_name}")
            updated_row = row.copy()
            updated_row['benefits_text'] = "Error: Invalid URL"
            updated_rows.append(updated_row)
            continue
            
        print(f"\n[{idx+1}/{total_to_process}] Processing: {scheme_name}")
        print(f"URL: {scheme_link}")
        
        # Try to extract benefits with retries and exponential backoff
        benefits_text = None
        retries = 0
        backoff_time = 2  # start with 2 seconds
        
        while benefits_text is None and retries < MAX_RETRIES:
            try:
                benefits_text = extract_benefits_section(driver, scheme_link)
                
                # Check if no benefits were found
                if not benefits_text or benefits_text == "No benefits information found":
                    print(f"Attempt {retries+1}: Could not find benefits")
                    retries += 1
                    
                    if retries < MAX_RETRIES:
                        print(f"Waiting {backoff_time} seconds before retry...")
                        time.sleep(backoff_time)
                        backoff_time *= 1.5  # Exponential backoff
                        benefits_text = None  # Reset to trigger retry
                    else:
                        benefits_text = "No benefits information found after multiple attempts"
                else:
                    print(f"Successfully extracted benefits: {len(benefits_text)} characters")
            except Exception as e:
                print(f"Attempt {retries+1} failed: {e}")
                retries += 1
                
                if retries < MAX_RETRIES:
                    print(f"Waiting {backoff_time} seconds before retry...")
                    time.sleep(backoff_time)
                    backoff_time *= 1.5  # Exponential backoff
                else:
                    benefits_text = f"Error: Failed to extract benefits after {MAX_RETRIES} attempts"
        
        # Update the row
        updated_row = row.copy()
        updated_row['benefits_text'] = benefits_text
        updated_rows.append(updated_row)
        
        # Save checkpoint every 5 schemes
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            # Create temporary DataFrame with processed rows
            processed_df = pd.DataFrame(updated_rows)
            # Merge with schemes that already have benefits
            temp_df = pd.concat([has_benefits_df, processed_df])
            temp_df.to_excel(CHECKPOINT_FILE, index=False)
            print(f"Checkpoint saved. Processed {idx+1}/{total_to_process} schemes without benefits.")
            
        # Add a random delay to avoid detection (longer delay for more robustness)
        delay_time = 2 + random.random() * 3
        print(f"Waiting {delay_time:.2f} seconds before next scheme...")
        time.sleep(delay_time)
    
    # Close the WebDriver
    driver.quit()
    
    # Combine updated rows with rows that already had benefits
    final_df = pd.concat([has_benefits_df, pd.DataFrame(updated_rows)])
    
    # Sort by original index to maintain the original order if available
    if 'original_index' in final_df.columns:
        final_df = final_df.sort_values('original_index')
    
    # Save final results
    final_df.to_excel(OUTPUT_FILE, index=False)
    
    print(f"\nExtraction complete! Updated {len(updated_rows)} schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the Eligibility section of each scheme

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from urllib.parse import urljoin

# Load existing Excel file with scheme names and links
df = pd.read_excel("myscheme_details_with_benefits.xlsx")

# Create a new column for eligibility information if it doesn't exist
if 'eligibility_text' not in df.columns:
    df['eligibility_text'] = ""

# Function to extract eligibility information from a scheme page
def extract_eligibility(url):
    try:
        # Modify URL to directly access the eligibility section
        base_url = url.split('#')[0] if '#' in url else url
        eligibility_url = base_url
        
        # If URL doesn't end with /eligibility, try appending it
        if not eligibility_url.endswith("/eligibility"):
            # Check for fadsp pattern shown in your example
            if "fadsp" in eligibility_url or "schemes" in eligibility_url:
                if not eligibility_url.endswith('/'):
                    eligibility_url += '/'
                eligibility_url += "#eligibility"
        
        # Send HTTP request with proper headers
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(eligibility_url, headers=headers)
        response.raise_for_status()
        
        # Parse HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Try to find eligibility section in various formats
        eligibility_section = None
        
        # Method 1: Find the section by heading
        for heading in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
            if "eligibility" in heading.text.strip().lower():
                eligibility_section = heading
                break
        
        eligibility_points = []
        
        if eligibility_section:
            # Get the parent section that might contain all eligibility info
            parent_section = eligibility_section.parent
            
            # Method 1: Try to find list items within the section
            if parent_section:
                list_items = parent_section.find_all('li')
                if list_items:
                    eligibility_points = [item.text.strip() for item in list_items]
            
            # Method 2: Look for paragraphs with Roman numeral patterns like (i), (ii), etc.
            if not eligibility_points:
                roman_pattern = re.compile(r'\((i{1,3}|iv|v|vi{1,3}|ix|x)\)')
                
                # Check all paragraphs after the eligibility heading
                next_elements = []
                current = eligibility_section.next_sibling
                
                # Collect siblings until we hit another heading or a maximum number of elements
                while current and len(next_elements) < 20:
                    if current.name and current.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                        break
                    if current.name in ['p', 'div'] and current.text.strip():
                        next_elements.append(current)
                    current = current.next_sibling
                
                # Check collected elements for Roman numeral patterns
                for element in next_elements:
                    if roman_pattern.search(element.text):
                        eligibility_points.append(element.text.strip())
                    elif element.text.strip() and len(element.text.strip()) > 20:  # Arbitrary minimum length for meaningful content
                        eligibility_points.append(element.text.strip())
            
            # Method 3: Look for standard numbered paragraphs (1., 2., etc.)
            if not eligibility_points:
                number_pattern = re.compile(r'^\d+\.')
                next_elements = eligibility_section.find_next_siblings(['p', 'div'])
                
                for element in next_elements:
                    text = element.text.strip()
                    if number_pattern.search(text):
                        eligibility_points.append(text)
                    elif text and len(text) > 20:  # Check for meaningful content
                        eligibility_points.append(text)
        
        # If still no eligibility points, try a broader search
        if not eligibility_points:
            # Look for any section that might contain eligibility info
            for section in soup.find_all(['div', 'section']):
                if "eligibility" in section.get('id', '').lower() or "eligibility" in section.get('class', [''])[0].lower() if section.get('class') else False:
                    # Check all paragraphs within this section
                    paragraphs = section.find_all(['p', 'div'])
                    for p in paragraphs:
                        if p.text.strip() and len(p.text.strip()) > 20:  # Check for meaningful content
                            eligibility_points.append(p.text.strip())
        
        # If still nothing, look for any paragraph that has keywords indicating eligibility criteria
        if not eligibility_points:
            eligibility_keywords = [
                "eligible to apply", "can apply", "applicant should", "applicant must",
                "requirements for", "criteria for", "qualification", "who can apply"
            ]
            
            for p in soup.find_all(['p', 'div', 'span']):
                text = p.text.strip()
                if any(keyword in text.lower() for keyword in eligibility_keywords) and len(text) > 20:
                    eligibility_points.append(text)
        
        # Remove duplicates while preserving order
        unique_eligibility_points = []
        seen_points = set()
        for point in eligibility_points:
            # Clean the point text to normalize it for comparison
            # Remove numbering and special characters for comparison
            cleaned_text = re.sub(r'^\d+\.\s*|\([ivx]+\)\s*', '', point).strip()
            # Only add if we haven't seen this text before
            if cleaned_text not in seen_points and cleaned_text:
                seen_points.add(cleaned_text)
                unique_eligibility_points.append(point)
        
        # Join all points into a single text with consistent numbering
        if unique_eligibility_points:
            # Clean up the points and ensure they have numbers
            cleaned_points = []
            for i, point in enumerate(unique_eligibility_points):
                # Remove existing numbers if they exist and add consistent numbering
                # This handles both numeric (1., 2.) and Roman numeral ((i), (ii)) formats
                point = re.sub(r'^\d+\.\s*|\([ivx]+\)\s*', '', point).strip()
                cleaned_points.append(f"{i+1}. {point}")
            
            return "\n".join(cleaned_points)
        else:
            return "No eligibility information found."
    
    except Exception as e:
        return f"Error extracting eligibility: {e}"

# Process each scheme in the dataframe
for index, row in df.iterrows():
    if pd.notna(row['scheme_link']):
        print(f"Processing {row['scheme_name']}...")
        
        # Extract eligibility information
        eligibility_info = extract_eligibility(row['scheme_link'])
        
        # Update the dataframe
        df.at[index, 'eligibility_text'] = eligibility_info
        
        # Pause to avoid overloading the server
        time.sleep(2)
    else:
        df.at[index, 'eligibility_text'] = "No link available"

# Save the updated dataframe to Excel
df.to_excel("myscheme_details_with_eligibility.xlsx", index=False)
print("Extraction completed and saved to myscheme_details_with_eligibility.xlsx")

In [ ]:
#Extracting the Exclusion section of each scheme

In [ ]:
import os
import time
import random
import pandas as pd
from tqdm import tqdm  # for progress reporting
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# File paths
INPUT_FILE = "myscheme_details_with_eligibility.xlsx"  # Your existing file with scheme links
OUTPUT_FILE = "myscheme_exlusions.xlsx"                # New file for scheme details (exclusions)
CHECKPOINT_FILE = "exlusions_checkpoint.xlsx"          # Checkpoint file for progress tracking

# Maximum number of retries for failed requests
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_input():
    """
    Load the input Excel file with scheme details and ensure an 'original_index' column exists.
    """
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found. Run the initial scraper first.")
        return None
    df = pd.read_excel(INPUT_FILE)
    if "original_index" not in df.columns:
        df["original_index"] = df.index
    return df

def load_checkpoint():
    """
    Load the checkpoint file to track progress.
    Returns a DataFrame and a set of processed scheme links.
    """
    if os.path.exists(CHECKPOINT_FILE):
        details_df = pd.read_excel(CHECKPOINT_FILE)
        if not details_df.empty and 'scheme_link' in details_df.columns:
            processed_links = set(details_df["scheme_link"])
            return details_df, processed_links
    columns = ["scheme_name", "scheme_link", "details_text", "original_index", "exclusions_text"]
    return pd.DataFrame(columns=columns), set()

def save_checkpoint(df):
    """
    Save current progress to the checkpoint file.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)
    print(f"Checkpoint saved with {len(df)} schemes.")

def extract_exclusions_section(driver, url):
    """
    Extract the 'Exclusions' section content from a scheme page.
    Attempts to capture all text in the Exclusions container (including numbered list items and additional notes).
    If not found, returns a default message.
    """
    try:
        driver.get(url)
        # Wait for the page to load (up to 15 seconds)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(2)  # Additional delay for JavaScript rendering

        # Attempt 1: Locate the Exclusions section by its ID
        try:
            exclusions_container = driver.find_element(By.ID, "exclusions")
        except NoSuchElementException:
            exclusions_container = None

        # Attempt 2: If not found by ID, try finding a header with "Exclusions"
        if not exclusions_container:
            headers = driver.find_elements(
                By.XPATH, 
                "//h2[contains(translate(text(),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'exclusions')]"
            )
            if headers:
                exclusions_container = headers[0].find_element(By.XPATH, "ancestor::div[1]")
        
        if exclusions_container:
            # Instead of extracting only the ordered list, get the full text content of the container.
            full_text = exclusions_container.text.strip()
            if full_text:
                return full_text
        
        # If no container or text is found, return a default message.
        return "Section Not Available against this scheme"
    except Exception as e:
        print(f"Error extracting Exclusions section from {url}: {e}")
        return "Section Not Available against this scheme"

def main():
    """
    Main function to process scheme links and extract the Exclusions section.
    It loads the input file, processes only those schemes that have not yet been processed,
    extracts the Exclusions section, and saves the data to an output Excel file.
    """
    input_df = load_input()
    if input_df is None:
        return
    print(f"Loaded {len(input_df)} schemes from {INPUT_FILE}.")
    
    checkpoint_df, processed_links = load_checkpoint()
    print(f"Already processed {len(processed_links)} scheme links from checkpoint.")
    
    # Create a new column for exclusions if it doesn't exist.
    if "exclusions_text" not in input_df.columns:
        input_df["exclusions_text"] = ""
    
    # Filter rows without exclusions information.
    no_exclusions_df = input_df[input_df["exclusions_text"].isna() | (input_df["exclusions_text"] == "")]
    has_exclusions_df = input_df[~(input_df["exclusions_text"].isna() | (input_df["exclusions_text"] == ""))]
    
    print(f"Found {len(no_exclusions_df)} schemes without exclusions information")
    print(f"{len(has_exclusions_df)} schemes already have exclusions information")
    
    if len(no_exclusions_df) == 0:
        print("No schemes need processing. Exiting.")
        return
    
    driver = setup_driver()
    updated_rows = []
    total_to_process = len(no_exclusions_df)
    
    for idx, row in tqdm(no_exclusions_df.iterrows(), total=total_to_process, desc="Processing schemes for Exclusions"):
        scheme_name = row["scheme_name"]
        scheme_link = row["scheme_link"]
        
        # Skip if URL is invalid
        if not scheme_link or not scheme_link.startswith("http"):
            print(f"Skipping invalid URL for: {scheme_name}")
            updated_row = row.copy()
            updated_row["exclusions_text"] = "Error: Invalid URL"
            updated_rows.append(updated_row)
            continue
        
        print(f"\nProcessing scheme: {scheme_name}")
        print(f"URL: {scheme_link}")
        
        exclusions_text = None
        retries = 0
        backoff_time = 2  # starting delay in seconds
        
        while exclusions_text is None and retries < MAX_RETRIES:
            try:
                exclusions_text = extract_exclusions_section(driver, scheme_link)
                if not exclusions_text or len(exclusions_text) < 10:
                    print(f"Attempt {retries+1}: Insufficient exclusions content for {scheme_name}")
                    retries += 1
                    if retries < MAX_RETRIES:
                        print(f"Waiting {backoff_time} seconds before retry...")
                        time.sleep(backoff_time)
                        backoff_time *= 1.5
                        exclusions_text = None  # Reset for retry
                    else:
                        exclusions_text = "Section Not Available against this scheme"
                else:
                    print(f"Successfully extracted exclusions: {len(exclusions_text)} characters")
            except Exception as e:
                print(f"Attempt {retries+1} failed for {scheme_name}: {e}")
                retries += 1
                if retries < MAX_RETRIES:
                    print(f"Waiting {backoff_time} seconds before retry...")
                    time.sleep(backoff_time)
                    backoff_time *= 1.5
                else:
                    exclusions_text = "Section Not Available against this scheme"
        
        updated_row = row.copy()
        updated_row["exclusions_text"] = exclusions_text
        updated_rows.append(updated_row)
        
        # Save checkpoint every 5 schemes or at the end
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            processed_df = pd.DataFrame(updated_rows)
            temp_df = pd.concat([has_exclusions_df, processed_df], ignore_index=True)
            temp_df.to_excel(CHECKPOINT_FILE, index=False)
            print(f"Checkpoint saved. Processed {idx+1}/{total_to_process} schemes without exclusions.")
        
        delay_time = 2 + random.random() * 3
        print(f"Waiting {delay_time:.2f} seconds before next scheme...")
        time.sleep(delay_time)
    
    driver.quit()
    
    final_df = pd.concat([has_exclusions_df, pd.DataFrame(updated_rows)], ignore_index=True)
    if "original_index" in final_df.columns:
        final_df = final_df.sort_values("original_index")
    final_df.to_excel(OUTPUT_FILE, index=False)
    
    print(f"\nExtraction complete! Updated {len(updated_rows)} schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the Application mode of each scheme

In [ ]:
import os
import time
import random
import pandas as pd
from tqdm import tqdm  # For progress reporting
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# File paths (adjust these paths as needed)
INPUT_FILE = "myscheme_exlusions.xlsx"  # Existing file with scheme_name, scheme_link, etc.
OUTPUT_FILE = "myscheme_application_mode.xlsx"         # Final output file with Application Mode info
CHECKPOINT_FILE = "application_mode_checkpoint.xlsx"    # Checkpoint file for progress

# Maximum number of retries for failed extraction attempts
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_input():
    """
    Load the input Excel file with scheme details and ensure an 'original_index' column exists.
    """
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found. Run the initial scraper first.")
        return None
    df = pd.read_excel(INPUT_FILE)
    if "original_index" not in df.columns:
        df["original_index"] = df.index
    return df

def load_checkpoint():
    """
    Load the checkpoint file to track progress.
    Returns a DataFrame and a set of processed scheme links.
    """
    if os.path.exists(CHECKPOINT_FILE):
        df = pd.read_excel(CHECKPOINT_FILE)
        if not df.empty and 'scheme_link' in df.columns:
            processed_links = set(df["scheme_link"])
            return df, processed_links
    # If file doesn't exist, start fresh with desired columns.
    columns = ["scheme_name", "scheme_link", "page", "application_mode", "original_index"]
    return pd.DataFrame(columns=columns), set()

def save_checkpoint(df):
    """
    Save current progress to the checkpoint file.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)
    print(f"Checkpoint saved with {len(df)} schemes.")

def extract_application_mode(driver, url):
    """
    Extract the Application Mode (Online, Offline, or both) from a scheme page.
    
    The function loads the page (using the original URL; note that the fragment part, e.g., "#application-process",
    is not necessary to load the content), scrolls to the "Application Process" section, and then searches for mode indicators.
    
    Returns a string listing available modes (e.g., "Online", "Offline", or "Offline, Online").
    If no mode is found, returns "No mode information found".
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(2)  # Additional delay for JS to render

        # Scroll to the Application Process section using its ID
        try:
            app_section = driver.find_element(By.ID, "application-process")
        except NoSuchElementException:
            # If the section is not explicitly marked with an id, try finding a heading
            headings = driver.find_elements(By.XPATH, "//h2[contains(translate(text(),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'application process')]")
            if headings:
                app_section = headings[0].find_element(By.XPATH, "ancestor::div[1]")
            else:
                return "Section Not Available against this scheme"

        driver.execute_script("arguments[0].scrollIntoView();", app_section)
        time.sleep(1)

        # Now, within the application process section, look for span elements that contain mode names.
        # We assume the mode names will be "Online" or "Offline" (case-insensitive).
        mode_elements = app_section.find_elements(By.XPATH, ".//span")
        modes_found = []
        for elem in mode_elements:
            text = elem.text.strip()
            if text.lower() in ["online", "offline"]:
                modes_found.append(text)
        
        if modes_found:
            # Remove duplicates and sort (so order is consistent)
            unique_modes = sorted(set(modes_found))
            return ", ".join(unique_modes)
        else:
            return "No mode information found"
    except Exception as e:
        print(f"Error extracting application mode from {url}: {e}")
        return "Error"

def main():
    """
    Main function to process scheme links and extract Application Mode information.
    It loads the input file, processes only those schemes that have not yet been processed,
    extracts the Application Mode from the Application Process section, and saves the data to an output Excel file.
    """
    input_df = load_input()
    if input_df is None:
        return
    print(f"Loaded {len(input_df)} schemes from {INPUT_FILE}.")

    checkpoint_df, processed_links = load_checkpoint()
    print(f"Already processed {len(processed_links)} scheme links from checkpoint.")

    # Create a new column for application_mode if it doesn't exist.
    if "application_mode" not in input_df.columns:
        input_df["application_mode"] = ""

    # Filter rows without application mode information.
    no_mode_df = input_df[input_df["application_mode"].isna() | (input_df["application_mode"] == "")]
    has_mode_df = input_df[~(input_df["application_mode"].isna() | (input_df["application_mode"] == ""))]

    print(f"Found {len(no_mode_df)} schemes without application mode information")
    print(f"{len(has_mode_df)} schemes already have application mode information")

    if len(no_mode_df) == 0:
        print("No schemes need processing. Exiting.")
        return

    driver = setup_driver()
    updated_rows = []
    total_to_process = len(no_mode_df)

    for idx, row in tqdm(no_mode_df.iterrows(), total=total_to_process, desc="Processing schemes for Application Mode"):
        scheme_name = row["scheme_name"]
        scheme_link = row["scheme_link"]

        # Skip if URL is invalid
        if not scheme_link or not scheme_link.startswith("http"):
            print(f"Skipping invalid URL for: {scheme_name}")
            updated_row = row.copy()
            updated_row["application_mode"] = "Error: Invalid URL"
            updated_rows.append(updated_row)
            continue

        print(f"\nProcessing scheme: {scheme_name}")
        print(f"URL: {scheme_link}")

        mode_text = None
        retries = 0
        backoff_time = 2  # Starting delay in seconds

        while mode_text is None and retries < MAX_RETRIES:
            try:
                mode_text = extract_application_mode(driver, scheme_link)
                if not mode_text or len(mode_text) < 3 or mode_text == "No mode information found":
                    print(f"Attempt {retries+1}: Insufficient application mode info for {scheme_name}")
                    retries += 1
                    if retries < MAX_RETRIES:
                        print(f"Waiting {backoff_time} seconds before retry...")
                        time.sleep(backoff_time)
                        backoff_time *= 1.5
                        mode_text = None  # Reset for retry
                    else:
                        mode_text = "Section Not Available against this scheme"
                else:
                    print(f"Successfully extracted application mode: {mode_text}")
            except Exception as e:
                print(f"Attempt {retries+1} failed for {scheme_name}: {e}")
                retries += 1
                if retries < MAX_RETRIES:
                    print(f"Waiting {backoff_time} seconds before retry...")
                    time.sleep(backoff_time)
                    backoff_time *= 1.5
                else:
                    mode_text = "Section Not Available against this scheme"

        updated_row = row.copy()
        updated_row["application_mode"] = mode_text
        updated_rows.append(updated_row)

        # Save checkpoint every 5 schemes or at the end
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            processed_df = pd.DataFrame(updated_rows)
            temp_df = pd.concat([has_mode_df, processed_df], ignore_index=True)
            temp_df.to_excel(CHECKPOINT_FILE, index=False)
            print(f"Checkpoint saved. Processed {idx+1}/{total_to_process} schemes without application mode.")
        
        delay_time = 2 + random.random() * 3
        print(f"Waiting {delay_time:.2f} seconds before next scheme...")
        time.sleep(delay_time)

    driver.quit()

    final_df = pd.concat([has_mode_df, pd.DataFrame(updated_rows)], ignore_index=True)
    if "original_index" in final_df.columns:
        final_df = final_df.sort_values("original_index")
    final_df.to_excel(OUTPUT_FILE, index=False)
    print(f"\nExtraction complete! Updated {len(updated_rows)} schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the Application process section of each scheme

In [ ]:
import os
import time
import random
import re
import pandas as pd
from tqdm import tqdm  # for progress reporting
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager

# File paths
INPUT_FILE = "myscheme_application_mode.xlsx"  # Your input file with scheme links, etc.
OUTPUT_FILE = "myscheme_application_process.xlsx"       # Final output file with Application Process details
CHECKPOINT_FILE = "application_process_checkpoint.xlsx"  # Checkpoint file to resume progress

# Maximum number of retries for failed extraction attempts
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_input():
    """
    Load the input Excel file with scheme details and ensure an 'original_index' column exists.
    """
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found. Run the initial scraper first.")
        return None
    df = pd.read_excel(INPUT_FILE)
    if "original_index" not in df.columns:
        df["original_index"] = df.index
    return df

def load_checkpoint():
    """
    Load the checkpoint file to track progress.
    Returns a DataFrame and a set of processed scheme links.
    """
    if os.path.exists(CHECKPOINT_FILE):
        df = pd.read_excel(CHECKPOINT_FILE)
        if not df.empty and 'scheme_link' in df.columns:
            processed_links = set(df["scheme_link"])
            return df, processed_links
    columns = ["scheme_name", "scheme_link", "page", "application_steps_offline", 
               "application_notes_offline", "application_steps_online", 
               "application_notes_online", "original_index"]
    return pd.DataFrame(columns=columns), set()

def save_checkpoint(df):
    """
    Save current progress to the checkpoint file.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)
    print(f"Checkpoint saved with {len(df)} schemes.")

def clean_application_text(text):
    """
    Clean the application process text by removing unwanted header text and tabs.
    """
    # Remove "Application Process" header if present
    text = re.sub(r"^Application Process\s*", "", text, flags=re.IGNORECASE)
    
    # Remove "Offline" or "Online" tabs if present
    text = re.sub(r"^(Offline|Online)\s*", "", text, flags=re.IGNORECASE)
    
    # Also handle case where they might be in the middle of text due to newlines
    text = re.sub(r"\n(Offline|Online)\s*", "\n", text, flags=re.IGNORECASE)
    
    # Remove duplicate "Application Process" subheadings
    text = re.sub(r"\nApplication Process\s*", "\n", text, flags=re.IGNORECASE)
    
    return text.strip()

def extract_content_from_current_tab(driver, container):
    """
    Extract application process content from the currently active tab.
    """
    try:
        # Try to get content from paragraphs
        paragraphs = container.find_elements(By.TAG_NAME, "p")
        if paragraphs:
            content_text = "\n".join([p.text.strip() for p in paragraphs if p.text.strip()])
        else:
            # Try to get numbered list items
            list_items = container.find_elements(By.XPATH, ".//ol/li | .//ul/li")
            if list_items:
                content_text = "\n".join([f"{i+1}. {item.text.strip()}" for i, item in enumerate(list_items) if item.text.strip()])
            else:
                # Fall back to all text
                content_text = container.text.strip()
                
        # Clean the text
        content_text = clean_application_text(content_text)
        
        # Better parsing of steps and notes
        steps = []
        notes = []
        
        # Split by lines to process step by step
        lines = content_text.split('\n')
        current_section = "steps"  # Default section
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
                
            # Check if this line is a note
            if line.lower().startswith("note:") or line.lower().startswith("note "):
                current_section = "notes"
                notes.append(line)
            # Check if this is a step indicator
            elif line.lower().startswith("step") or re.match(r"^\d+\.", line):
                current_section = "steps"
                steps.append(line)
            # Otherwise add to current section
            else:
                if current_section == "notes":
                    notes.append(line)
                else:
                    steps.append(line)
        
        steps_text = "\n".join(steps).strip()
        notes_text = "\n".join(notes).strip()
        
        if not notes_text:
            notes_text = "No specific notes found"
            
        return {"steps": steps_text, "notes": notes_text}
    except Exception as e:
        print(f"Error extracting content from tab: {e}")
        return {"steps": "Error extracting content", "notes": "Error extracting content"}

def extract_application_process_section(driver, url):
    """
    Extract the 'Application Process' section from a scheme page,
    including both online and offline modes if available.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(3)  # Allow extra time for JavaScript rendering
        
        # Attempt to locate the Application Process section by its ID.
        try:
            app_proc_container = driver.find_element(By.ID, "application-process")
        except NoSuchElementException:
            # If not found by ID, try finding a heading with "Application Process"
            headings = driver.find_elements(By.XPATH, 
                "//h2[contains(translate(text(),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'application process')]")
            if headings:
                app_proc_container = headings[0].find_element(By.XPATH, "ancestor::div[2]")
            else:
                # Try to find content in div with markdown content
                try:
                    markdown_divs = driver.find_elements(By.XPATH, "//div[contains(@class, 'markdown') or contains(@data-slate-node, 'value')]")
                    if markdown_divs:
                        app_proc_container = markdown_divs[0]
                    else:
                        return {
                            "offline_steps": "Section Not Available", 
                            "offline_notes": "Section Not Available",
                            "online_steps": "Section Not Available",
                            "online_notes": "Section Not Available"
                        }
                except NoSuchElementException:
                    return {
                        "offline_steps": "Section Not Available", 
                        "offline_notes": "Section Not Available",
                        "online_steps": "Section Not Available",
                        "online_notes": "Section Not Available"
                    }
        
        driver.execute_script("arguments[0].scrollIntoView();", app_proc_container)
        time.sleep(2)
        
        # Initialize result dictionary
        result = {
            "offline_steps": "Section Not Available",
            "offline_notes": "Section Not Available",
            "online_steps": "Section Not Available",
            "online_notes": "Section Not Available"
        }
        
        # Look for tabs (Offline/Online)
        tabs = driver.find_elements(By.XPATH, "//div[contains(@class, 'px-4') and .//span[contains(text(), 'Offline') or contains(text(), 'Online')]]")
        
        # If tabs exist, process each tab separately
        if tabs and len(tabs) > 1:
            # First, extract content from the currently active tab
            active_tab_text = tabs[0].find_element(By.TAG_NAME, "span").text.strip().lower()
            active_content = extract_content_from_current_tab(driver, app_proc_container)
            
            if "offline" in active_tab_text:
                result["offline_steps"] = active_content["steps"]
                result["offline_notes"] = active_content["notes"]
            elif "online" in active_tab_text:
                result["online_steps"] = active_content["steps"]
                result["online_notes"] = active_content["notes"]
            
            # Now click on the other tab(s) and extract their content
            for tab in tabs[1:]:
                try:
                    tab_text = tab.find_element(By.TAG_NAME, "span").text.strip().lower()
                    
                    # Skip if we've already processed this tab type
                    if ("offline" in tab_text and result["offline_steps"] != "Section Not Available") or \
                       ("online" in tab_text and result["online_steps"] != "Section Not Available"):
                        continue
                    
                    # Click the tab
                    driver.execute_script("arguments[0].scrollIntoView();", tab)
                    driver.execute_script("arguments[0].click();", tab)
                    time.sleep(2)  # Wait for content to load
                    
                    # Extract content from the newly active tab
                    tab_content = extract_content_from_current_tab(driver, app_proc_container)
                    
                    # Store content based on tab type
                    if "offline" in tab_text:
                        result["offline_steps"] = tab_content["steps"]
                        result["offline_notes"] = tab_content["notes"]
                    elif "online" in tab_text:
                        result["online_steps"] = tab_content["steps"]
                        result["online_notes"] = tab_content["notes"]
                        
                except (ElementClickInterceptedException, NoSuchElementException) as e:
                    print(f"Error clicking or processing tab: {e}")
                    continue
        else:
            # No tabs found, extract content normally
            content = extract_content_from_current_tab(driver, app_proc_container)
            
            # Determine if content refers to online or offline process
            content_text = content["steps"].lower()
            if "online" in content_text or "website" in content_text or "portal" in content_text or "internet" in content_text:
                result["online_steps"] = content["steps"]
                result["online_notes"] = content["notes"]
            else:
                result["offline_steps"] = content["steps"]
                result["offline_notes"] = content["notes"]
        
        return result
    except Exception as e:
        print(f"Error extracting Application Process section from {url}: {e}")
        return {
            "offline_steps": "Error extracting section", 
            "offline_notes": "Error extracting section",
            "online_steps": "Error extracting section",
            "online_notes": "Error extracting section"
        }

def main():
    """
    Main function to process scheme links and extract the Application Process section.
    It loads the input file, processes only those schemes that have not yet been processed,
    extracts the Application Process section for both online and offline modes,
    and saves the data to an output Excel file.
    """
    input_df = load_input()
    if input_df is None:
        return
    print(f"Loaded {len(input_df)} schemes from {INPUT_FILE}.")
    
    checkpoint_df, processed_links = load_checkpoint()
    print(f"Already processed {len(processed_links)} scheme links from checkpoint.")
    
    # Create new columns for application process if they don't exist
    new_columns = [
        "application_steps_offline", "application_notes_offline", 
        "application_steps_online", "application_notes_online"
    ]
    
    for col in new_columns:
        if col not in input_df.columns:
            input_df[col] = ""
    
    # Filter rows without application process info
    no_app_df = input_df[
        (input_df["application_steps_offline"].isna() | (input_df["application_steps_offline"] == "")) &
        (input_df["application_steps_online"].isna() | (input_df["application_steps_online"] == ""))
    ]
    
    has_app_df = input_df[
        ~((input_df["application_steps_offline"].isna() | (input_df["application_steps_offline"] == "")) &
          (input_df["application_steps_online"].isna() | (input_df["application_steps_online"] == "")))
    ]
    
    print(f"Found {len(no_app_df)} schemes without Application Process information")
    print(f"{len(has_app_df)} schemes already have Application Process information")
    
    if len(no_app_df) == 0:
        print("No schemes need processing. Exiting.")
        return
    
    driver = setup_driver()
    updated_rows = []
    total_to_process = len(no_app_df)
    
    for idx, row in tqdm(no_app_df.iterrows(), total=total_to_process, desc="Processing schemes for Application Process"):
        scheme_name = row["scheme_name"]
        scheme_link = row["scheme_link"]
        
        # Skip if URL is invalid
        if not scheme_link or not scheme_link.startswith("http"):
            print(f"Skipping invalid URL for: {scheme_name}")
            updated_row = row.copy()
            updated_row["application_steps_offline"] = "Error: Invalid URL"
            updated_row["application_notes_offline"] = "Error: Invalid URL"
            updated_row["application_steps_online"] = "Error: Invalid URL"
            updated_row["application_notes_online"] = "Error: Invalid URL"
            updated_rows.append(updated_row)
            continue
        
        print(f"\nProcessing scheme: {scheme_name}")
        print(f"URL: {scheme_link}")
        
        app_proc_data = None
        retries = 0
        backoff_time = 2
        
        while app_proc_data is None and retries < MAX_RETRIES:
            try:
                app_proc_data = extract_application_process_section(driver, scheme_link)
                
                # Check if we got sufficient content from either mode
                offline_content_len = len(app_proc_data["offline_steps"]) if app_proc_data["offline_steps"] != "Section Not Available" else 0
                online_content_len = len(app_proc_data["online_steps"]) if app_proc_data["online_steps"] != "Section Not Available" else 0
                
                if offline_content_len < 10 and online_content_len < 10:
                    print(f"Attempt {retries+1}: Insufficient Application Process content for {scheme_name}")
                    retries += 1
                    if retries < MAX_RETRIES:
                        print(f"Waiting {backoff_time} seconds before retry...")
                        time.sleep(backoff_time)
                        backoff_time *= 1.5
                        app_proc_data = None
                    else:
                        app_proc_data = {
                            "offline_steps": "Section Not Available against this scheme", 
                            "offline_notes": "Section Not Available against this scheme",
                            "online_steps": "Section Not Available against this scheme",
                            "online_notes": "Section Not Available against this scheme"
                        }
                else:
                    print(f"Successfully extracted Application Process for {scheme_name}")
            except Exception as e:
                print(f"Attempt {retries+1} failed for {scheme_name}: {e}")
                retries += 1
                if retries < MAX_RETRIES:
                    print(f"Waiting {backoff_time} seconds before retry...")
                    time.sleep(backoff_time)
                    backoff_time *= 1.5
                else:
                    app_proc_data = {
                        "offline_steps": "Section Not Available against this scheme", 
                        "offline_notes": "Section Not Available against this scheme",
                        "online_steps": "Section Not Available against this scheme",
                        "online_notes": "Section Not Available against this scheme"
                    }
        
        updated_row = row.copy()
        updated_row["application_steps_offline"] = app_proc_data["offline_steps"]
        updated_row["application_notes_offline"] = app_proc_data["offline_notes"]
        updated_row["application_steps_online"] = app_proc_data["online_steps"]
        updated_row["application_notes_online"] = app_proc_data["online_notes"]
        updated_rows.append(updated_row)
        
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            processed_df = pd.DataFrame(updated_rows)
            # Make sure all columns exist in both DataFrames before concatenation
            for col in processed_df.columns:
                if col not in has_app_df.columns:
                    has_app_df[col] = ""
            for col in has_app_df.columns:
                if col not in processed_df.columns:
                    processed_df[col] = ""
                    
            temp_df = pd.concat([has_app_df, processed_df], ignore_index=True)
            temp_df.to_excel(CHECKPOINT_FILE, index=False)
            print(f"Checkpoint saved. Processed {idx+1}/{total_to_process} schemes without Application Process info.")
        
        delay_time = 2 + random.random() * 3
        print(f"Waiting {delay_time:.2f} seconds before next scheme...")
        time.sleep(delay_time)
    
    driver.quit()
    
    # Make sure all columns exist in both DataFrames before concatenation
    for col in pd.DataFrame(updated_rows).columns:
        if col not in has_app_df.columns:
            has_app_df[col] = ""
            
    final_df = pd.concat([has_app_df, pd.DataFrame(updated_rows)], ignore_index=True)
    if "original_index" in final_df.columns:
        final_df = final_df.sort_values("original_index")
    final_df.to_excel(OUTPUT_FILE, index=False)
    
    print(f"\nExtraction complete! Updated {len(updated_rows)} schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the Documents required section of each scheme

In [ ]:
import os
import time
import random
import re
import pandas as pd
from tqdm import tqdm  # for progress reporting
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# File paths
INPUT_FILE = "myscheme_application_process.xlsx"  # Your input file with scheme links, etc.
OUTPUT_FILE = "myscheme_documents_required.xlsx"  # Final output file with Documents Required details
CHECKPOINT_FILE = "documents_required_checkpoint.xlsx"  # Checkpoint file to resume progress

# Maximum number of retries for failed extraction attempts
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_input():
    """
    Load the input Excel file with scheme details and ensure an 'original_index' column exists.
    """
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found. Run the initial scraper first.")
        return None
    df = pd.read_excel(INPUT_FILE)
    if "original_index" not in df.columns:
        df["original_index"] = df.index
    return df

def load_checkpoint():
    """
    Load the checkpoint file to track progress.
    Returns a DataFrame and a set of processed scheme links.
    """
    if os.path.exists(CHECKPOINT_FILE):
        df = pd.read_excel(CHECKPOINT_FILE)
        if not df.empty and 'scheme_link' in df.columns:
            processed_links = set(df["scheme_link"])
            return df, processed_links
    # Define required columns for documents required extraction.
    columns = ["scheme_name", "scheme_link", "page", "documents_required", "original_index"]
    return pd.DataFrame(columns=columns), set()

def save_checkpoint(df):
    """
    Save current progress to the checkpoint file.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)
    print(f"Checkpoint saved with {len(df)} schemes.")

def extract_documents_required(driver, url):
    """
    Extract the 'Documents Required' section content from a scheme page.
    Attempts multiple methods:
      - First, it looks for a container with id "documents-required".
      - If not found, it looks for a heading containing "Documents Required"
        and then takes an ancestor container.
      - It then attempts to extract numbered list items, or if that fails, all text.
    Returns the formatted text or a default message if the section isn’t found.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(3)  # Allow time for JS to render
        
        # Attempt 1: Look for the container by ID.
        try:
            docs_container = driver.find_element(By.ID, "documents-required")
        except NoSuchElementException:
            docs_container = None
        
        # Attempt 2: If not found by ID, try finding a heading with "Documents Required"
        if not docs_container:
            headings = driver.find_elements(
                By.XPATH, 
                "//h3[contains(translate(text(),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'documents required')]"
            )
            if headings:
                # Use an ancestor container; adjust the level as needed.
                docs_container = headings[0].find_element(By.XPATH, "ancestor::div[2]")
            else:
                return "Documents Required section not found on this page"
        
        driver.execute_script("arguments[0].scrollIntoView();", docs_container)
        time.sleep(1)
        
        documents_list = []
        
        # Method 1: Try to extract numbered items from div elements with data-slate-node="element"
        doc_items = docs_container.find_elements(By.XPATH, ".//div[@data-slate-node='element']")
        if doc_items:
            for item in doc_items:
                text = item.text.strip()
                if text and re.match(r'^\d+\.', text):
                    documents_list.append(text)
        
        # Method 2: If Method 1 didn't work, look for list items.
        if not documents_list:
            list_items = docs_container.find_elements(By.XPATH, ".//ol/li | .//ul/li")
            if list_items:
                for i, item in enumerate(list_items):
                    text = item.text.strip()
                    if text:
                        documents_list.append(f"{i+1}. {text}")
        
        # Method 3: If still nothing, try to get all paragraph text.
        if not documents_list:
            paragraphs = docs_container.find_elements(By.TAG_NAME, "p")
            for p in paragraphs:
                text = p.text.strip()
                if text:
                    documents_list.append(text)
        
        # Method 4: If nothing so far, look for spans with data-slate-string.
        if not documents_list:
            spans = docs_container.find_elements(By.XPATH, ".//span[@data-slate-string='true']")
            for span in spans:
                text = span.text.strip()
                if text:
                    documents_list.append(text)
        
        # Final fallback: get all text from container.
        if not documents_list:
            all_text = docs_container.text.strip()
            if all_text:
                # Optionally, split by newline and remove the heading if present.
                lines = re.split(r'\n+', all_text)
                for line in lines:
                    cleaned = line.strip()
                    if cleaned and not cleaned.lower().startswith("documents required"):
                        documents_list.append(cleaned)
        
        if documents_list:
            return "\n".join(documents_list)
        else:
            return "No document requirements found in this section"
    
    except Exception as e:
        print(f"Error extracting Documents Required section from {url}: {e}")
        return "Error extracting Documents Required section"

def main():
    """
    Main function to process scheme links and extract the Documents Required section.
    It loads the input file, processes only those schemes that have not yet been processed,
    extracts the Documents Required section from each scheme, and saves the data to an output Excel file.
    """
    input_df = load_input()
    if input_df is None:
        return
    print(f"Loaded {len(input_df)} schemes from {INPUT_FILE}.")
    
    checkpoint_df, processed_links = load_checkpoint()
    print(f"Already processed {len(processed_links)} scheme links from checkpoint.")
    
    # Create a new column for documents_required if it doesn't exist.
    if "documents_required" not in input_df.columns:
        input_df["documents_required"] = ""
    
    # Filter rows without documents_required info.
    no_docs_df = input_df[input_df["documents_required"].isna() | (input_df["documents_required"] == "")]
    has_docs_df = input_df[~(input_df["documents_required"].isna() | (input_df["documents_required"] == ""))]
    
    print(f"Found {len(no_docs_df)} schemes without Documents Required information")
    print(f"{len(has_docs_df)} schemes already have Documents Required information")
    
    if len(no_docs_df) == 0:
        print("No schemes need processing. Exiting.")
        return
    
    driver = setup_driver()
    updated_rows = []
    total_to_process = len(no_docs_df)
    
    for idx, row in tqdm(no_docs_df.iterrows(), total=total_to_process, desc="Processing schemes for Documents Required"):
        scheme_name = row["scheme_name"]
        scheme_link = row["scheme_link"]
        
        # Skip invalid URLs.
        if not scheme_link or not scheme_link.startswith("http"):
            print(f"Skipping invalid URL for: {scheme_name}")
            updated_row = row.copy()
            updated_row["documents_required"] = "Error: Invalid URL"
            updated_rows.append(updated_row)
            continue
        
        print(f"\nProcessing scheme: {scheme_name}")
        print(f"URL: {scheme_link}")
        
        docs_text = None
        retries = 0
        backoff_time = 2
        
        while docs_text is None and retries < MAX_RETRIES:
            try:
                docs_text = extract_documents_required(driver, scheme_link)
                if not docs_text or len(docs_text) < 10:
                    print(f"Attempt {retries+1}: Insufficient Documents Required content for {scheme_name}")
                    retries += 1
                    if retries < MAX_RETRIES:
                        print(f"Waiting {backoff_time} seconds before retry...")
                        time.sleep(backoff_time)
                        backoff_time *= 1.5
                        docs_text = None
                    else:
                        docs_text = "Section Not Available against this scheme"
                else:
                    print(f"Successfully extracted Documents Required for {scheme_name}")
            except Exception as e:
                print(f"Attempt {retries+1} failed for {scheme_name}: {e}")
                retries += 1
                if retries < MAX_RETRIES:
                    print(f"Waiting {backoff_time} seconds before retry...")
                    time.sleep(backoff_time)
                    backoff_time *= 1.5
                else:
                    docs_text = "Section Not Available against this scheme"
        
        updated_row = row.copy()
        updated_row["documents_required"] = docs_text
        updated_rows.append(updated_row)
        
        # Save checkpoint every 5 schemes or at the end.
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            processed_df = pd.DataFrame(updated_rows)
            temp_df = pd.concat([has_docs_df, processed_df], ignore_index=True)
            temp_df.to_excel(CHECKPOINT_FILE, index=False)
            print(f"Checkpoint saved. Processed {idx+1}/{total_to_process} schemes.")
        
        delay_time = 2 + random.random() * 3
        print(f"Waiting {delay_time:.2f} seconds before next scheme...")
        time.sleep(delay_time)
    
    driver.quit()
    
    final_df = pd.concat([has_docs_df, pd.DataFrame(updated_rows)], ignore_index=True)
    if "original_index" in final_df.columns:
        final_df = final_df.sort_values("original_index")
    final_df.to_excel(OUTPUT_FILE, index=False)
    print(f"\nExtraction complete! Updated {len(updated_rows)} new schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the Scheme type section of each scheme

In [ ]:
import os
import time
import random
import pandas as pd
import logging
from tqdm import tqdm  # for progress reporting
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

# This will download (if needed) and return the path of the latest chromedriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Now you can use your driver as usual
driver.get("https://www.google.com")
print("Latest chromedriver version is being used.")
driver.quit()

# File paths (adjust as needed)
INPUT_FILE = "myscheme_documents_required.xlsx"  # Your input file with scheme_name, scheme_link, etc.
OUTPUT_FILE = "myscheme_scheme_type.xlsx"         # Final output file with Scheme Type information
CHECKPOINT_FILE = "scheme_type_checkpoint.xlsx"   # Checkpoint file to resume progress

# Maximum number of retries for failed extraction attempts
MAX_RETRIES = 3

# List of Indian state names (expand as needed)
STATE_NAMES = [
    "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", "Goa", 
    "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", 
    "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland", 
    "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura", 
    "Uttar Pradesh", "Uttarakhand", "West Bengal", "Delhi", "Puducherry", "Chandigarh", 
    "Andaman and Nicobar Islands", "Dadra and Nagar Haveli and Daman and Diu"
]

# Configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[logging.StreamHandler()])

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_input():
    """
    Load the input Excel file with scheme details and ensure an 'original_index' column exists.
    """
    if not os.path.exists(INPUT_FILE):
        logging.error(f"Error: {INPUT_FILE} not found. Run the initial scraper first.")
        return None
    df = pd.read_excel(INPUT_FILE)
    if "original_index" not in df.columns:
        df["original_index"] = df.index
    return df

def load_checkpoint():
    """
    Load the checkpoint file to track progress.
    Returns a DataFrame and a set of processed scheme links.
    """
    if os.path.exists(CHECKPOINT_FILE):
        df = pd.read_excel(CHECKPOINT_FILE)
        if not df.empty and 'scheme_link' in df.columns:
            processed_links = set(df["scheme_link"])
            return df, processed_links
    columns = ["scheme_name", "scheme_link", "page", "scheme_type", "original_index"]
    return pd.DataFrame(columns=columns), set()

def save_checkpoint(df):
    """
    Save current progress to the checkpoint file.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)
    logging.info(f"Checkpoint saved with {len(df)} schemes.")

def extract_scheme_type(driver, url):
    """
    Extract the scheme type from a scheme page.
    
    This function loads the page, scrolls to a likely area where the scheme type is displayed,
    and then searches for known state names. If any state name is found (case-insensitive),
    it returns "State Scheme"; otherwise, it returns "Central Scheme".
    
    If no information is found, it returns "Scheme Type Not Available".
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(3)  # Allow JS to render
        
        # Strategy 1: Try to find an <h3> element that might display the scheme type.
        try:
            type_elem = driver.find_element(By.XPATH, "//h3[contains(@class, 'text-') and contains(@class, 'cursor-pointer')]")
            type_text = type_elem.text.strip()
            logging.debug(f"Found scheme type text via h3: {type_text}")
        except NoSuchElementException:
            type_text = ""
        
        # Strategy 2: If not found, try to find any element that contains "Ministry"
        if not type_text:
            try:
                type_elem = driver.find_element(By.XPATH, "//div[contains(text(),'Ministry')]")
                type_text = type_elem.text.strip()
                logging.debug(f"Found scheme type text via div: {type_text}")
            except NoSuchElementException:
                type_text = ""
        
        # Strategy 3: If still not found, get body text and search for state names
        if not type_text:
            body_text = driver.find_element(By.TAG_NAME, "body").text.strip()
            type_text = body_text[:200]  # take first 200 chars as a guess
        
        if type_text:
            # Check if any state name is in the text
            for state in STATE_NAMES:
                if state.lower() in type_text.lower():
                    logging.info(f"Scheme type determined as State Scheme based on state name: {state}")
                    return "State Scheme"
            # If no state name found, assume central scheme.
            logging.info("Scheme type determined as Central Scheme")
            return "Central Scheme"
        else:
            return "Scheme Type Not Available"
    except Exception as e:
        logging.error(f"Error extracting scheme type from {url}: {e}")
        return "Error extracting scheme type"

def main():
    """
    Main function to process scheme links and extract the Scheme Type.
    Loads the input file, skips already processed schemes (via checkpoint),
    extracts the Scheme Type for each scheme, and saves the results to an output Excel file.
    """
    input_df = load_input()
    if input_df is None:
        return
    logging.info(f"Loaded {len(input_df)} schemes from {INPUT_FILE}.")
    
    checkpoint_df, processed_links = load_checkpoint()
    logging.info(f"Already processed {len(processed_links)} scheme links from checkpoint.")
    
    # Process only unprocessed schemes based on unique scheme_link
    unprocessed_df = input_df[~input_df["scheme_link"].isin(processed_links)]
    logging.info(f"Processing {len(unprocessed_df)} new schemes for Scheme Type extraction.")
    
    updated_rows = []
    total_to_process = len(unprocessed_df)
    driver = setup_driver()
    
    for idx, row in tqdm(unprocessed_df.iterrows(), total=total_to_process, desc="Processing Scheme Type"):
        scheme_name = row["scheme_name"]
        scheme_link = row["scheme_link"]
        
        if not scheme_link or not scheme_link.startswith("http"):
            logging.warning(f"Skipping invalid URL for: {scheme_name}")
            updated_row = row.copy()
            updated_row["scheme_type"] = "Error: Invalid URL"
            updated_rows.append(updated_row)
            continue
        
        logging.info(f"\nProcessing scheme: {scheme_name}")
        logging.info(f"URL: {scheme_link}")
        
        scheme_type = None
        retries = 0
        backoff_time = 2
        
        while scheme_type is None and retries < MAX_RETRIES:
            try:
                scheme_type = extract_scheme_type(driver, scheme_link)
                if not scheme_type or len(scheme_type) < 3 or "Not Available" in scheme_type:
                    logging.warning(f"Attempt {retries+1}: Insufficient scheme type info for {scheme_name}")
                    retries += 1
                    if retries < MAX_RETRIES:
                        logging.info(f"Waiting {backoff_time} seconds before retry...")
                        time.sleep(backoff_time)
                        backoff_time *= 1.5
                        scheme_type = None
                    else:
                        scheme_type = "Section Not Available against this scheme"
                else:
                    logging.info(f"Successfully extracted scheme type: {scheme_type}")
            except Exception as e:
                logging.error(f"Attempt {retries+1} failed for {scheme_name}: {e}")
                retries += 1
                if retries < MAX_RETRIES:
                    logging.info(f"Waiting {backoff_time} seconds before retry...")
                    time.sleep(backoff_time)
                    backoff_time *= 1.5
                else:
                    scheme_type = "Section Not Available against this scheme"
        
        updated_row = row.copy()
        updated_row["scheme_type"] = scheme_type
        updated_rows.append(updated_row)
        
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            processed_df = pd.DataFrame(updated_rows)
            temp_df = pd.concat([checkpoint_df, processed_df], ignore_index=True)
            save_checkpoint(temp_df)
        
        delay_time = 2 + random.random() * 3
        logging.info(f"Waiting {delay_time:.2f} seconds before next scheme...")
        time.sleep(delay_time)
    
    driver.quit()
    
    final_df = pd.concat([checkpoint_df, pd.DataFrame(updated_rows)], ignore_index=True)
    if "original_index" in final_df.columns:
        final_df = final_df.sort_values("original_index")
    final_df.to_excel(OUTPUT_FILE, index=False)
    logging.info(f"\nExtraction complete! Updated {len(updated_rows)} new schemes.")
    logging.info(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the FAQ section of each scheme

In [ ]:
import os
import time
import random
import pandas as pd
from tqdm import tqdm  # for progress reporting
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager

# File paths (adjust as needed)
INPUT_FILE = "myscheme_scheme_type.xlsx"  # Your input file with scheme_name, scheme_link, etc.
OUTPUT_FILE = "myscheme_faqs.xlsx"              # Final output file with FAQ details
CHECKPOINT_FILE = "faqs_checkpoint.xlsx"        # Checkpoint file to resume progress

# Maximum number of retries for failed extraction attempts
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_input():
    """
    Load the input Excel file with scheme details and ensure an 'original_index' column exists.
    """
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found. Run the initial scraper first.")
        return None
    df = pd.read_excel(INPUT_FILE)
    if "original_index" not in df.columns:
        df["original_index"] = df.index
    return df

def load_checkpoint():
    """
    Load the checkpoint file to track progress.
    Returns a DataFrame and a set of processed scheme links.
    """
    if os.path.exists(CHECKPOINT_FILE):
        df = pd.read_excel(CHECKPOINT_FILE)
        if not df.empty and 'scheme_link' in df.columns:
            processed_links = set(df["scheme_link"])
            return df, processed_links
    columns = ["scheme_name", "scheme_link", "page", "faqs", "original_index"]
    return pd.DataFrame(columns=columns), set()

def save_checkpoint(df):
    """
    Save current progress to the checkpoint file.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)
    print(f"Checkpoint saved with {len(df)} schemes.")

def extract_faqs_section(driver, url):
    """
    Extract the Frequently Asked Questions (FAQs) section from a scheme page.
    
    This function loads the given URL, scrolls to the FAQ container (with id="faqs"),
    and then iterates over FAQ items. For each FAQ, it:
      - Extracts the question text (from a <p> element that appears to contain the question)
      - Checks if the answer container (a sibling <div> with a class containing "rounded-b" or similar)
        is hidden; if it is, it simulates a click on the question to reveal the answer.
      - Extracts the answer text.
    
    The function returns a single string that contains a numbered list of FAQs in the format:
      1. Q: <question>
         A: <answer>
      2. Q: <question>
         A: <answer>
    If the section is not found, returns "FAQs section not available".
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(3)  # allow extra time for JS to render
        
        try:
            faq_container = driver.find_element(By.ID, "faqs")
        except NoSuchElementException:
            return "FAQs section not available"
        
        driver.execute_script("arguments[0].scrollIntoView();", faq_container)
        time.sleep(2)
        
        # Assume FAQ items are within a grid container inside #faqs.
        # Adjust the XPath if needed.
        faq_items = faq_container.find_elements(By.XPATH, ".//div[contains(@class, 'py-4')]")
        
        if not faq_items:
            return "No FAQs found"
        
        faq_list = []
        question_counter = 1
        
        for item in faq_items:
            try:
                # Look for a question element (e.g., a <p> with bold text)
                question_elem = item.find_element(By.XPATH, ".//p[contains(@class, 'font-bold')]")
                question_text = question_elem.text.strip()
            except NoSuchElementException:
                question_text = ""
            
            # Look for the answer container.
            # In your HTML sample, the answer appears in a sibling div (with classes like "rounded-b" etc.)
            try:
                answer_elem = item.find_element(By.XPATH, ".//div[contains(@class, 'rounded-b')]")
                # If the answer element has class "hidden", click the question to reveal it.
                if "hidden" in answer_elem.get_attribute("class").lower():
                    try:
                        question_elem.click()
                        time.sleep(1)
                    except Exception as e:
                        print(f"Could not click question to reveal answer: {e}")
                answer_text = answer_elem.text.strip()
            except NoSuchElementException:
                answer_text = ""
            
            # If answer text is empty, try to fetch any text from the item.
            if not answer_text:
                answer_text = item.text.strip()
            
            # If we have a valid question and answer, add to the FAQ list.
            if question_text:
                faq_list.append(f"{question_counter}. Q: {question_text}\n   A: {answer_text}")
                question_counter += 1
        
        if faq_list:
            return "\n\n".join(faq_list)
        else:
            return "No FAQs found"
    
    except Exception as e:
        print(f"Error extracting FAQs from {url}: {e}")
        return "Error extracting FAQs"

def main():
    """
    Main function to process scheme links and extract the FAQs section.
    It loads the input file, processes only those schemes that haven't been processed yet,
    extracts the FAQs from each scheme, and saves the results to an output Excel file.
    """
    input_df = load_input()
    if input_df is None:
        return
    print(f"Loaded {len(input_df)} schemes from {INPUT_FILE}.")
    
    checkpoint_df, processed_links = load_checkpoint()
    print(f"Already processed {len(processed_links)} scheme links from checkpoint.")
    
    # Process only schemes not in the checkpoint
    unprocessed_df = input_df[~input_df["scheme_link"].isin(processed_links)]
    print(f"Processing {len(unprocessed_df)} new schemes for FAQs extraction.")
    
    # Create a new column for FAQs if it doesn't exist.
    if "faqs" not in input_df.columns:
        input_df["faqs"] = ""
    
    updated_rows = []
    total_to_process = len(unprocessed_df)
    driver = setup_driver()
    
    for idx, row in tqdm(unprocessed_df.iterrows(), total=total_to_process, desc="Processing FAQs"):
        scheme_name = row["scheme_name"]
        scheme_link = row["scheme_link"]
        
        if not scheme_link or not scheme_link.startswith("http"):
            print(f"Skipping invalid URL for: {scheme_name}")
            updated_row = row.copy()
            updated_row["faqs"] = "Error: Invalid URL"
            updated_rows.append(updated_row)
            continue
        
        print(f"\nProcessing scheme: {scheme_name}")
        print(f"URL: {scheme_link}")
        
        faqs_text = None
        retries = 0
        backoff_time = 2
        
        while faqs_text is None and retries < MAX_RETRIES:
            try:
                faqs_text = extract_faqs_section(driver, scheme_link)
                if not faqs_text or len(faqs_text) < 10:
                    print(f"Attempt {retries+1}: Insufficient FAQs content for {scheme_name}")
                    retries += 1
                    if retries < MAX_RETRIES:
                        print(f"Waiting {backoff_time} seconds before retry...")
                        time.sleep(backoff_time)
                        backoff_time *= 1.5
                        faqs_text = None
                    else:
                        faqs_text = "Section Not Available against this scheme"
                else:
                    print(f"Successfully extracted FAQs for {scheme_name}")
            except Exception as e:
                print(f"Attempt {retries+1} failed for {scheme_name}: {e}")
                retries += 1
                if retries < MAX_RETRIES:
                    print(f"Waiting {backoff_time} seconds before retry...")
                    time.sleep(backoff_time)
                    backoff_time *= 1.5
                else:
                    faqs_text = "Section Not Available against this scheme"
        
        updated_row = row.copy()
        updated_row["faqs"] = faqs_text
        updated_rows.append(updated_row)
        
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            processed_df = pd.DataFrame(updated_rows)
            temp_df = pd.concat([checkpoint_df, processed_df], ignore_index=True)
            temp_df.to_excel(CHECKPOINT_FILE, index=False)
            print(f"Checkpoint saved. Processed {idx+1}/{total_to_process} schemes.")
        
        delay_time = 2 + random.random() * 3
        print(f"Waiting {delay_time:.2f} seconds before next scheme...")
        time.sleep(delay_time)
    
    driver.quit()
    
    final_df = pd.concat([checkpoint_df, pd.DataFrame(updated_rows)], ignore_index=True)
    if "original_index" in final_df.columns:
        final_df = final_df.sort_values("original_index")
    final_df.to_excel(OUTPUT_FILE, index=False)
    print(f"\nExtraction complete! Updated {len(updated_rows)} new schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the sources section of each scheme

In [ ]:
import os
import time
import random
import pandas as pd
from tqdm import tqdm  # For progress reporting
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# File paths
INPUT_FILE = "myscheme_faqs.xlsx"  # Your input file with scheme_name, scheme_link, etc.
OUTPUT_FILE = "myscheme_scheme_sources.xlsx"    # Final output file with Sources And References details
CHECKPOINT_FILE = "scheme_sources_checkpoint.xlsx"  # Checkpoint file to resume progress

# Maximum number of retries for failed extraction attempts
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_input():
    """
    Load the input Excel file with scheme details and ensure an 'original_index' column exists.
    """
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found. Run the initial scraper first.")
        return None
    df = pd.read_excel(INPUT_FILE)
    if "original_index" not in df.columns:
        df["original_index"] = df.index
    return df

def load_checkpoint():
    """
    Load the checkpoint file to track progress.
    Returns a DataFrame and a set of processed scheme links.
    """
    if os.path.exists(CHECKPOINT_FILE):
        df = pd.read_excel(CHECKPOINT_FILE)
        if not df.empty and 'scheme_link' in df.columns:
            processed_links = set(df["scheme_link"])
            return df, processed_links
    columns = ["scheme_name", "scheme_link", "page", "sources", "original_index"]
    return pd.DataFrame(columns=columns), set()

def save_checkpoint(df):
    """
    Save current progress to the checkpoint file.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)
    print(f"Checkpoint saved with {len(df)} schemes.")

def extract_sources_section(driver, url):
    """
    Extract the 'Sources And References' section from a scheme page.
    
    This function loads the given URL, scrolls to the section (identified by id="sources" or
    by a heading containing "Sources And References"), and then attempts to extract the sources.
    
    It searches within the container for anchor (<a>) tags (which are typically links) and/or paragraph (<p>) tags.
    For each source found, it numbers the source and, if an anchor tag is present, appends the URL in parentheses.
    
    If no source information is found, it returns a default message.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(3)  # Allow extra time for JS to render
        
        # Attempt 1: Locate the Sources container by ID
        try:
            sources_container = driver.find_element(By.ID, "sources")
        except NoSuchElementException:
            sources_container = None
        
        # Attempt 2: If not found by ID, try finding a heading with "Sources And References"
        if not sources_container:
            headings = driver.find_elements(By.XPATH,
                "//h3[contains(translate(text(),'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz'),'sources and references')]"
            )
            if headings:
                sources_container = headings[0].find_element(By.XPATH, "ancestor::div[1]")
            else:
                return "Sources And References section not available"
        
        driver.execute_script("arguments[0].scrollIntoView();", sources_container)
        time.sleep(2)
        
        sources_list = []
        
        # Method 1: Look for anchor tags within the container.
        anchors = sources_container.find_elements(By.TAG_NAME, "a")
        if anchors:
            for i, a in enumerate(anchors):
                link_text = a.text.strip()
                href = a.get_attribute("href")
                # Sometimes the anchor might be empty, so try to get the parent text if available.
                if not link_text:
                    link_text = a.find_element(By.XPATH, "./..").text.strip()
                if link_text or href:
                    sources_list.append(f"{i+1}. {link_text} ({href})")
        
        # Method 2: If no anchors or if the list is empty, look for paragraph elements.
        if not sources_list:
            paragraphs = sources_container.find_elements(By.TAG_NAME, "p")
            if paragraphs:
                for i, p in enumerate(paragraphs):
                    text = p.text.strip()
                    if text:
                        sources_list.append(f"{i+1}. {text}")
        
        # Method 3: Fallback - get all text and try to split by newlines
        if not sources_list:
            all_text = sources_container.text.strip()
            if all_text:
                lines = [line.strip() for line in all_text.split("\n") if line.strip()]
                for i, line in enumerate(lines):
                    # Skip lines that are part of the heading if needed.
                    sources_list.append(f"{i+1}. {line}")
        
        if sources_list:
            return "\n".join(sources_list)
        else:
            return "No Sources And References information found"
    
    except Exception as e:
        print(f"Error extracting Sources And References section from {url}: {e}")
        return "Error extracting Sources And References section"

def main():
    """
    Main function to process scheme links and extract the Sources And References section.
    It loads the input file, processes only schemes not already processed (using checkpoint data),
    extracts the Sources section for each scheme, and saves the results to an output Excel file.
    """
    input_df = load_input()
    if input_df is None:
        return
    print(f"Loaded {len(input_df)} schemes from {INPUT_FILE}.")
    
    checkpoint_df, processed_links = load_checkpoint()
    print(f"Already processed {len(processed_links)} scheme links from checkpoint.")
    
    # Process only unprocessed schemes based on scheme_link uniqueness
    unprocessed_df = input_df[~input_df["scheme_link"].isin(processed_links)]
    print(f"Processing {len(unprocessed_df)} new schemes for Sources And References extraction.")
    
    # Create a new column for sources if it doesn't exist.
    if "sources" not in input_df.columns:
        input_df["sources"] = ""
    
    updated_rows = []
    total_to_process = len(unprocessed_df)
    driver = setup_driver()
    
    for idx, row in tqdm(unprocessed_df.iterrows(), total=total_to_process, desc="Processing Sources"):
        scheme_name = row["scheme_name"]
        scheme_link = row["scheme_link"]
        
        if not scheme_link or not scheme_link.startswith("http"):
            print(f"Skipping invalid URL for: {scheme_name}")
            updated_row = row.copy()
            updated_row["sources"] = "Error: Invalid URL"
            updated_rows.append(updated_row)
            continue
        
        print(f"\nProcessing scheme: {scheme_name}")
        print(f"URL: {scheme_link}")
        
        sources_text = None
        retries = 0
        backoff_time = 2
        
        while sources_text is None and retries < MAX_RETRIES:
            try:
                sources_text = extract_sources_section(driver, scheme_link)
                if not sources_text or len(sources_text) < 10:
                    print(f"Attempt {retries+1}: Insufficient Sources content for {scheme_name}")
                    retries += 1
                    if retries < MAX_RETRIES:
                        print(f"Waiting {backoff_time} seconds before retry...")
                        time.sleep(backoff_time)
                        backoff_time *= 1.5
                        sources_text = None
                    else:
                        sources_text = "Section Not Available against this scheme"
                else:
                    print(f"Successfully extracted Sources And References for {scheme_name}")
            except Exception as e:
                print(f"Attempt {retries+1} failed for {scheme_name}: {e}")
                retries += 1
                if retries < MAX_RETRIES:
                    print(f"Waiting {backoff_time} seconds before retry...")
                    time.sleep(backoff_time)
                    backoff_time *= 1.5
                else:
                    sources_text = "Section Not Available against this scheme"
        
        updated_row = row.copy()
        updated_row["sources"] = sources_text
        updated_rows.append(updated_row)
        
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            processed_df = pd.DataFrame(updated_rows)
            temp_df = pd.concat([checkpoint_df, processed_df], ignore_index=True)
            save_checkpoint(temp_df)
            print(f"Checkpoint saved. Processed {idx+1}/{total_to_process} schemes.")
        
        delay_time = 2 + random.random() * 3
        print(f"Waiting {delay_time:.2f} seconds before next scheme...")
        time.sleep(delay_time)
    
    driver.quit()
    
    final_df = pd.concat([checkpoint_df, pd.DataFrame(updated_rows)], ignore_index=True)
    if "original_index" in final_df.columns:
        final_df = final_df.sort_values("original_index")
    final_df.to_excel(OUTPUT_FILE, index=False)
    print(f"\nExtraction complete! Updated {len(updated_rows)} new schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()

In [ ]:
#Extracting the Eligibility Criteria section of each scheme

In [ ]:
import os
import time
import random
import pandas as pd
from tqdm import tqdm  # for progress reporting
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

# File paths
INPUT_FILE = "myscheme_scheme_sources.xlsx"       # Your input file with scheme details (scheme_name, scheme_link, etc.)
OUTPUT_FILE = "myscheme_full_details.xlsx"    # Final output file with Eligibility Criteria data
CHECKPOINT_FILE = "eligibility_checkpoint.xlsx"       # Checkpoint file to resume progress

# Maximum number of retries for failed extraction attempts
MAX_RETRIES = 3

def setup_driver():
    """
    Configure and return a headless Chrome WebDriver.
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/91.0.4472.124 Safari/537.36"
    )
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def load_input():
    """
    Load the input Excel file with scheme details and ensure an 'original_index' column exists.
    """
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found. Run the initial scraper first.")
        return None
    df = pd.read_excel(INPUT_FILE)
    if "original_index" not in df.columns:
        df["original_index"] = df.index
    return df

def load_checkpoint():
    """
    Load the checkpoint file to track progress.
    Returns a DataFrame and a set of processed scheme links.
    """
    if os.path.exists(CHECKPOINT_FILE):
        df = pd.read_excel(CHECKPOINT_FILE)
        if not df.empty and 'scheme_link' in df.columns:
            processed_links = set(df["scheme_link"])
            return df, processed_links
    columns = ["scheme_name", "scheme_link", "page", "eligibility_criteria", "original_index"]
    return pd.DataFrame(columns=columns), set()

def save_checkpoint(df):
    """
    Save current progress to the checkpoint file.
    """
    df.to_excel(CHECKPOINT_FILE, index=False)
    print(f"Checkpoint saved with {len(df)} schemes.")

def extract_eligibility_criteria(driver, url):
    """
    Extract the eligibility criteria from a scheme page.
    Ensures no duplicate criteria are returned.
    
    Returns a string of eligibility criteria or "No eligibility criteria found" if none exists.
    """
    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        time.sleep(3)  # Allow JS to render
        
        # Initialize an empty set to store unique criteria
        criteria_set = set()
        
        # Try multiple approaches to find eligibility criteria
        
        # Approach 1: Find the main container first
        try:
            containers = driver.find_elements(By.XPATH, 
                "//div[contains(@class, 'grid grid-cols-1 md:flex flex-wrap gap-4')]")
            
            for container in containers:
                criteria_divs = container.find_elements(By.XPATH, ".//div[@title]")
                for div in criteria_divs:
                    title = div.get_attribute('title')
                    if title and title.strip():
                        criteria_set.add(title.strip())
        except Exception as e:
            print(f"Approach 1 failed: {e}")
        
        # Approach 2: Direct search for divs with specific classes and title attributes
        try:
            criteria_divs = driver.find_elements(
                By.XPATH, 
                "//div[contains(@class, 'border-green') and @title]"
            )
            
            for div in criteria_divs:
                title = div.get_attribute('title')
                if title and title.strip():
                    criteria_set.add(title.strip())
        except Exception as e:
            print(f"Approach 2 failed: {e}")
            
        # If criteria were found, join them into a comma-separated string
        if criteria_set:
            return ", ".join(sorted(criteria_set))  # Sort for consistent output
        else:
            return "No eligibility criteria found"
    
    except Exception as e:
        print(f"Error extracting eligibility criteria from {url}: {e}")
        return "Error extracting eligibility criteria"

def main():
    """
    Main function to process scheme links and extract the Scheme Type and Eligibility Criteria.
    """
    input_df = load_input()
    if input_df is None:
        return
    print(f"Loaded {len(input_df)} schemes from {INPUT_FILE}.")
    
    checkpoint_df, processed_links = load_checkpoint()
    print(f"Already processed {len(processed_links)} scheme links from checkpoint.")
    
    # Create new columns if they don't exist
    if "scheme_type" not in input_df.columns:
        input_df["scheme_type"] = ""
    if "eligibility_criteria" not in input_df.columns:
        input_df["eligibility_criteria"] = ""
    
    # Process only schemes that haven't been processed
    unprocessed_df = input_df[~input_df["scheme_link"].isin(processed_links)]
    print(f"Processing {len(unprocessed_df)} new schemes for extraction.")
    
    updated_rows = []
    total_to_process = len(unprocessed_df)
    driver = setup_driver()
    
    for idx, row in tqdm(unprocessed_df.iterrows(), total=total_to_process, desc="Processing Schemes"):
        scheme_name = row["scheme_name"]
        scheme_link = row["scheme_link"]
        
        if not scheme_link or not scheme_link.startswith("http"):
            print(f"Skipping invalid URL for: {scheme_name}")
            updated_row = row.copy()
            updated_row["scheme_type"] = "Error: Invalid URL"
            updated_row["eligibility_criteria"] = "Error: Invalid URL"
            updated_rows.append(updated_row)
            continue
        
        print(f"\nProcessing scheme: {scheme_name}")
        print(f"URL: {scheme_link}")
        
        # Extract scheme type with retries
        scheme_type = None
        eligibility_criteria = None
        retries = 0
        backoff_time = 2
        
        while (scheme_type is None or eligibility_criteria is None) and retries < MAX_RETRIES:
            try:
                if scheme_type is None:
                    scheme_type = extract_scheme_type(driver, scheme_link)
                
                if eligibility_criteria is None:
                    # Add a small delay before extracting eligibility criteria
                    time.sleep(1)
                    eligibility_criteria = extract_eligibility_criteria(driver, scheme_link)
                
                # Check if both extractions were successful
                if (not scheme_type or len(scheme_type) < 3 or "Not Available" in scheme_type or
                    not eligibility_criteria or "No eligibility" in eligibility_criteria or "Error" in eligibility_criteria):
                    print(f"Attempt {retries+1}: Insufficient info for {scheme_name}")
                    retries += 1
                    if retries < MAX_RETRIES:
                        print(f"Waiting {backoff_time} seconds before retry...")
                        time.sleep(backoff_time)
                        backoff_time *= 1.5
                        if "Not Available" in scheme_type or len(scheme_type) < 3:
                            scheme_type = None
                        if "No eligibility" in eligibility_criteria or "Error" in eligibility_criteria:
                            eligibility_criteria = None
                    else:
                        if scheme_type is None:
                            scheme_type = "Section Not Available against this scheme"
                        if eligibility_criteria is None:
                            eligibility_criteria = "No eligibility criteria found"
                else:
                    print(f"Successfully extracted scheme type: {scheme_type}")
                    print(f"Successfully extracted eligibility criteria: {eligibility_criteria}")
            except Exception as e:
                print(f"Attempt {retries+1} failed for {scheme_name}: {e}")
                retries += 1
                if retries < MAX_RETRIES:
                    print(f"Waiting {backoff_time} seconds before retry...")
                    time.sleep(backoff_time)
                    backoff_time *= 1.5
                else:
                    if scheme_type is None:
                        scheme_type = "Section Not Available against this scheme"
                    if eligibility_criteria is None:
                        eligibility_criteria = "No eligibility criteria found"
        
        updated_row = row.copy()
        updated_row["scheme_type"] = scheme_type
        updated_row["eligibility_criteria"] = eligibility_criteria
        updated_rows.append(updated_row)
        
        if (idx + 1) % 5 == 0 or (idx + 1) == total_to_process:
            processed_df = pd.DataFrame(updated_rows)
            temp_df = pd.concat([checkpoint_df, processed_df], ignore_index=True)
            temp_df.to_excel(CHECKPOINT_FILE, index=False)
            print(f"Checkpoint saved. Processed {idx+1}/{total_to_process} schemes.")
        
        delay_time = 2 + random.random() * 3
        print(f"Waiting {delay_time:.2f} seconds before next scheme...")
        time.sleep(delay_time)
    
    driver.quit()
    
    final_df = pd.concat([checkpoint_df, pd.DataFrame(updated_rows)], ignore_index=True)
    if "original_index" in final_df.columns:
        final_df = final_df.sort_values("original_index")
    final_df.to_excel(OUTPUT_FILE, index=False)
    print(f"\nExtraction complete! Updated {len(updated_rows)} new schemes.")
    print(f"Results saved to {OUTPUT_FILE}")

if __name__ == "__main__":
    main()